In [520]:
!pwd

/data/home/darina.bocharova


In [521]:
!ls

'Create spark session.ipynb'   lab01.json    lab02.json    sber-spark-ds-10
 lab01.ipynb		       lab02.ipynb   lab03.ipynb


In [522]:
import os
import sys

os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [523]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "ML app") 

spark = SparkSession.builder.config(conf=conf).appName("Spark ML app").getOrCreate()

In [524]:
spark

In [525]:
!hdfs dfs -ls /labs/slaba03

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2022-01-06 18:46 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2022-01-06 18:46 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2022-01-06 18:46 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2022-01-06 18:46 /labs/slaba03/laba03_views_programmes.csv


# raw_train

In [636]:
!hdfs dfs -head /labs/slaba03/laba03_train.csv

user_id,item_id,purchase
1654,74107,0
1654,89249,0
1654,99982,0
1654,89901,0
1654,100504,0
1654,66187,0
1654,84350,0
1654,92854,0
1654,72811,0
1654,86876,0
1654,102657,0
1654,100482,0
1654,89677,0
1654,99419,0
1654,66603,0
1654,7363,0
1654,1320,0
1654,88892,0
1654,66671,0
1654,75925,0
1654,92365,0
1654,101531,0
1654,100484,0
1654,83584,0
1654,88541,0
1654,11044,0
1654,11368,0
1654,72968,0
1654,72171,0
1654,100715,0
1654,100483,0
1654,95776,0
1654,87174,0
1654,74660,0
1654,88930,0
1654,88706,0
1654,94853,0
1654,94609,0
1654,10502,0
1654,80900,0
1654,9640,0
1654,10931,0
1654,98692,0
1654,88793,0
1654,79826,0
1654,88980,0
1654,85656,0
1654,92494,0
1654,66558,0
1654,95724,0
1654,66211,0
1654,91304,0
1654,84387,0
1654,74535,0
1654,10039,0
1654,92806,0
1654,100239,0
1654,79455,0
1654,95791,0
1654,92127,0
1654,65906,0
1654,74293,0
1654,95022,0
1654,74556,0
1654,89629,0
1654,87311,0
1654,93574,0
1654,99966,0
1654,72788,0
1654,10419,0
1654,74630,0
1654,88830,0
1654,94040,0
1654,92968,0
1654,828

In [637]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType, FloatType

In [638]:
schema_train = StructType([
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("purchase", IntegerType())
])

In [639]:
raw_train = spark.read.csv("/labs/slaba03/laba03_train.csv", sep=",", schema=schema_train, header=True)

In [640]:
raw_train.rdd.getNumPartitions()

5

In [641]:
    raw_train = raw_train.repartition(10)

In [642]:
raw_train.rdd.getNumPartitions()

10

In [643]:
raw_train.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- purchase: integer (nullable = true)



In [644]:
raw_train.show()

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
| 839229|  73173|       0|
| 851240|  99015|       0|
| 873015|  75459|       0|
| 868783|  93625|       0|
| 866207|    396|       0|
| 866396|  93476|       0|
| 865423|   5084|       0|
| 838937|  94739|       0|
| 866581|  10255|       0|
| 868415|   8486|       0|
| 871411|  95803|       0|
| 841676|  92933|       0|
| 838316|  88930|       0|
| 862319|  93069|       0|
| 842883|  93725|       0|
| 870927|  11508|       0|
| 868827|  11367|       0|
| 843338|  74431|       0|
| 865231|  77732|       0|
| 852464|  92524|       0|
+-------+-------+--------+
only showing top 20 rows



In [645]:
raw_train.count()

5032624

In [646]:
import pyspark.sql.functions as f

In [647]:
raw_train.filter(f.isnan("purchase")).count()

0

In [648]:
raw_train.filter(f.isnan("purchase")).count()

0

In [649]:
raw_train.select('user_id').distinct().count()

1941

In [650]:
user_id_list = raw_train.select('user_id').distinct().collect()

train_user_id_list = []
[train_user_id_list.append(user_id_list[i][0]) for i in range(len(user_id_list))]
train_user_id_list = set(train_user_id_list)

In [651]:
raw_train.select('item_id').distinct().count()

3704

In [652]:
item_id_list = raw_train.select('item_id').distinct().collect()

train_item_id_list = []
[train_item_id_list.append(item_id_list[i][0]) for i in range(len(item_id_list))]
train_item_id_list = set(train_item_id_list)

In [653]:
raw_train.filter(raw_train.user_id == 791395).show()

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
| 791395|  94685|       0|
| 791395|  93551|       0|
| 791395|  72900|       0|
| 791395|  72977|       0|
| 791395|  99871|       0|
| 791395|  99955|       0|
| 791395|  66769|       0|
| 791395|  66928|       0|
| 791395|  93061|       0|
| 791395|  89656|       0|
| 791395|  11519|       0|
| 791395|  91368|       0|
| 791395|  88549|       0|
| 791395|  74282|       0|
| 791395|  92862|       0|
| 791395| 101976|       0|
| 791395|  83659|       0|
| 791395|  10677|       0|
| 791395|  93638|       0|
| 791395|  78250|       0|
+-------+-------+--------+
only showing top 20 rows



In [654]:
raw_train.filter(raw_train.user_id == 791395).count()

2597

In [655]:
raw_train.filter(raw_train.user_id == 791395).groupBy("purchase").count().show()

+--------+-----+
|purchase|count|
+--------+-----+
|       1|    1|
|       0| 2596|
+--------+-----+



In [656]:
raw_train.filter(raw_train.user_id == 791395).groupBy("purchase").count().collect()

[Row(purchase=1, count=1), Row(purchase=0, count=2596)]

In [657]:
raw_train.filter(raw_train.item_id == 94685).show()

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
| 896756|  94685|       0|
| 896180|  94685|       0|
| 895064|  94685|       0|
| 892101|  94685|       0|
| 878614|  94685|       0|
| 874376|  94685|       0|
| 890136|  94685|       0|
| 882394|  94685|       0|
| 885354|  94685|       0|
| 892523|  94685|       0|
| 893739|  94685|       0|
| 896954|  94685|       0|
| 885512|  94685|       0|
| 894577|  94685|       0|
| 894093|  94685|       0|
| 892812|  94685|       0|
| 876221|  94685|       0|
| 888843|  94685|       0|
| 894258|  94685|       0|
| 879056|  94685|       0|
+-------+-------+--------+
only showing top 20 rows



In [658]:
raw_train.filter(raw_train.item_id == 94685).count()

1352

In [659]:
raw_train.filter(raw_train.item_id == 94685).groupBy("purchase").count().show()

+--------+-----+
|purchase|count|
+--------+-----+
|       1|    1|
|       0| 1351|
+--------+-----+



In [660]:
raw_train.filter(raw_train.item_id == 94685).groupBy("purchase").count().collect()

[Row(purchase=1, count=1), Row(purchase=0, count=1351)]

In [661]:
raw_train.groupBy("purchase").count().collect()

[Row(purchase=1, count=10904), Row(purchase=0, count=5021720)]

In [662]:
10904 / (10904 + 5021720)

0.0021666629575346776

## user_id feature

In [663]:
raw_train.groupBy("user_id").count().show(1941)

+-------+-----+
|user_id|count|
+-------+-----+
| 776188| 2603|
| 761341| 2580|
| 833685| 2533|
| 825061| 2589|
| 780033| 2578|
| 798454| 2604|
| 754230| 2611|
| 901457| 2629|
| 879401| 2568|
| 928140| 2584|
| 927211| 2553|
| 870928| 2606|
| 846231| 2599|
| 851486| 2623|
| 867850| 2611|
| 793876| 2577|
| 822709| 2639|
| 795620| 2592|
| 824008| 2617|
| 899993| 2571|
| 900203| 2619|
| 890476| 2589|
| 880451| 2603|
| 937345| 2597|
| 851848| 2572|
| 903826| 2590|
| 916566| 2604|
| 903491| 2583|
| 892290| 2591|
| 883098| 2566|
| 746713| 2553|
| 566758| 2655|
| 816426| 2603|
| 936359| 2627|
| 920599| 2570|
| 900335| 2600|
| 886063| 2594|
| 899224| 2564|
| 901323| 2600|
| 932980| 2592|
| 928231| 2637|
| 939459| 2566|
| 913290| 2564|
| 928490| 2599|
| 876234| 2611|
| 834532| 2620|
| 819462| 2601|
| 894860| 2632|
| 861601| 2560|
| 842784| 2581|
| 861707| 2541|
| 765780| 2566|
| 866581| 2513|
| 848246| 2569|
| 928002| 2607|
| 923662| 2626|
| 902359| 2567|
| 901042| 2622|
| 910808| 2523|
| 797350

In [664]:
total_user_id_purchase = raw_train.groupBy("user_id").count().withColumnRenamed('count', 'total_user_id_purchase')

In [665]:
total_user_id_purchase.show()

+-------+----------------------+
|user_id|total_user_id_purchase|
+-------+----------------------+
| 870928|                  2606|
| 846231|                  2599|
| 851486|                  2623|
| 867850|                  2611|
| 928140|                  2584|
| 927211|                  2553|
| 776188|                  2603|
| 761341|                  2580|
| 833685|                  2533|
| 825061|                  2589|
| 780033|                  2578|
| 798454|                  2604|
| 754230|                  2611|
| 879401|                  2568|
| 901457|                  2629|
| 851848|                  2572|
| 937345|                  2597|
| 793876|                  2577|
| 822709|                  2639|
| 795620|                  2592|
+-------+----------------------+
only showing top 20 rows



In [666]:
total_user_id_purchase.count()

1941

In [667]:
raw_train.filter(raw_train.purchase == 1).groupBy("user_id").count().show(1941)

+-------+-----+
|user_id|count|
+-------+-----+
| 879401|   11|
| 833685|   19|
| 825061|    5|
| 754230|   72|
| 761341|    1|
| 846231|    5|
| 776188|    3|
| 780033|    2|
| 870928|    2|
| 867850|    1|
| 928140|    1|
| 798454|    1|
| 927211|    1|
| 880451|   24|
| 900203|    9|
| 795620|   11|
| 793876|    5|
| 937345|    1|
| 899993|    1|
| 824008|    1|
| 822709|    1|
| 851848|    1|
| 903826|    4|
| 903491|    3|
| 883098|    5|
| 892290|    2|
| 566758|    1|
| 916566|    1|
| 936359|    2|
| 886063|    7|
| 900335|   12|
| 920599|    4|
| 932980|    1|
| 899224|    2|
| 939459|    1|
| 901323|    1|
| 928231|    2|
| 834532|    3|
| 928490|    4|
| 913290|    1|
| 819462|    1|
| 894860|    1|
| 765780|    5|
| 861601|   36|
| 861707|    2|
| 848246|   15|
| 740836|    7|
| 910808|    6|
| 866581|    4|
| 902359|    6|
| 901042|    1|
| 928002|    1|
| 787896|    9|
| 844427|    9|
| 842883|   16|
| 814235|    2|
| 778082|    8|
| 860986|    2|
| 849754|    1|
| 887904

In [668]:
user_id_purchase = raw_train.filter(raw_train.purchase == 1).groupBy("user_id").count().withColumnRenamed('count', 'user_id_purchase')

In [669]:
user_id_purchase.show()

+-------+----------------+
|user_id|user_id_purchase|
+-------+----------------+
| 833685|              19|
| 825061|               5|
| 754230|              72|
| 879401|              11|
| 928140|               1|
| 780033|               2|
| 870928|               2|
| 846231|               5|
| 776188|               3|
| 798454|               1|
| 927211|               1|
| 867850|               1|
| 761341|               1|
| 880451|              24|
| 900203|               9|
| 795620|              11|
| 899993|               1|
| 937345|               1|
| 793876|               5|
| 824008|               1|
+-------+----------------+
only showing top 20 rows



In [670]:
user_id_purchase.count()

1675

In [671]:
user_id_feature = total_user_id_purchase.join(user_id_purchase, on="user_id", how="outer")

In [672]:
user_id_feature.show()

+-------+----------------------+----------------+
|user_id|total_user_id_purchase|user_id_purchase|
+-------+----------------------+----------------+
| 754230|                  2611|              72|
| 761341|                  2580|               1|
| 776188|                  2603|               3|
| 780033|                  2578|               2|
| 798454|                  2604|               1|
| 825061|                  2589|               5|
| 833685|                  2533|              19|
| 846231|                  2599|               5|
| 851486|                  2623|            null|
| 867850|                  2611|               1|
| 870928|                  2606|               2|
| 879401|                  2568|              11|
| 901457|                  2629|            null|
| 927211|                  2553|               1|
| 928140|                  2584|               1|
| 793876|                  2577|               5|
| 795620|                  2592|              11|


In [673]:
user_id_feature.count()

1941

In [674]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col, pandas_udf, PandasUDFType

@f.pandas_udf(FloatType(), PandasUDFType.SCALAR)
def del_purchase(a, b):
      return a / b

In [675]:
active_user_purchase = user_id_feature.select("user_id", del_purchase("user_id_purchase", "total_user_id_purchase").alias("active_user_purchase"))

In [676]:
active_user_purchase.show()

+-------+--------------------+
|user_id|active_user_purchase|
+-------+--------------------+
| 754230|         0.027575642|
| 761341|         3.875969E-4|
| 776188|        0.0011525163|
| 780033|         7.757952E-4|
| 798454|        3.8402458E-4|
| 825061|        0.0019312475|
| 833685|         0.007500987|
| 846231|        0.0019238169|
| 851486|                null|
| 867850|        3.8299503E-4|
| 870928|         7.674597E-4|
| 879401|         0.004283489|
| 901457|                null|
| 927211|        3.9169606E-4|
| 928140|         3.869969E-4|
| 793876|        0.0019402406|
| 795620|         0.004243827|
| 822709|         3.789314E-4|
| 824008|        3.8211694E-4|
| 851848|         3.888025E-4|
+-------+--------------------+
only showing top 20 rows



In [677]:
active_user_purchase.count()

1941

In [678]:
active_user_purchase.filter(f.isnull("active_user_purchase")).count()

266

In [679]:
raw_train.show()

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
| 839229|  73173|       0|
| 851240|  99015|       0|
| 873015|  75459|       0|
| 868783|  93625|       0|
| 866207|    396|       0|
| 866396|  93476|       0|
| 865423|   5084|       0|
| 838937|  94739|       0|
| 866581|  10255|       0|
| 868415|   8486|       0|
| 871411|  95803|       0|
| 841676|  92933|       0|
| 838316|  88930|       0|
| 862319|  93069|       0|
| 842883|  93725|       0|
| 870927|  11508|       0|
| 868827|  11367|       0|
| 843338|  74431|       0|
| 865231|  77732|       0|
| 852464|  92524|       0|
+-------+-------+--------+
only showing top 20 rows



In [680]:
df_train = raw_train.join(active_user_purchase, on="user_id", how="outer")

In [681]:
df_train.show()

+-------+-------+--------+--------------------+
|user_id|item_id|purchase|active_user_purchase|
+-------+-------+--------+--------------------+
| 754230|   2360|       0|         0.027575642|
| 754230|  93608|       0|         0.027575642|
| 754230|  73409|       0|         0.027575642|
| 754230|  88960|       0|         0.027575642|
| 754230|  74382|       0|         0.027575642|
| 754230|  77347|       0|         0.027575642|
| 754230|  95051|       0|         0.027575642|
| 754230|  82243|       0|         0.027575642|
| 754230|  95785|       0|         0.027575642|
| 754230|  10453|       0|         0.027575642|
| 754230|  72712|       0|         0.027575642|
| 754230|  91813|       0|         0.027575642|
| 754230|  66724|       0|         0.027575642|
| 754230|  73283|       0|         0.027575642|
| 754230|  74526|       0|         0.027575642|
| 754230|  90743|       0|         0.027575642|
| 754230|  79897|       0|         0.027575642|
| 754230|  95846|       0|         0.027

In [682]:
df_train.filter(df_train.user_id == 833685).show()

+-------+-------+--------+--------------------+
|user_id|item_id|purchase|active_user_purchase|
+-------+-------+--------+--------------------+
| 833685|  85940|       0|         0.007500987|
| 833685|   8652|       1|         0.007500987|
| 833685|  79897|       0|         0.007500987|
| 833685|  94624|       0|         0.007500987|
| 833685|  74951|       0|         0.007500987|
| 833685|  95813|       0|         0.007500987|
| 833685|  93616|       0|         0.007500987|
| 833685|    782|       0|         0.007500987|
| 833685|  93624|       0|         0.007500987|
| 833685|  79030|       0|         0.007500987|
| 833685|  78248|       0|         0.007500987|
| 833685|  73685|       0|         0.007500987|
| 833685|  95326|       0|         0.007500987|
| 833685|  74295|       0|         0.007500987|
| 833685|  10475|       0|         0.007500987|
| 833685|  99010|       0|         0.007500987|
| 833685|  94989|       0|         0.007500987|
| 833685|  92358|       0|         0.007

## item_id feature

In [683]:
raw_train.groupBy("item_id").count().show(3704)

+-------+-----+
|item_id|count|
+-------+-----+
|  74820| 1344|
|  90019| 1315|
|   8638| 1379|
|  99817| 1364|
|  93486| 1401|
|  95080| 1376|
|  74757| 1359|
|  95940| 1409|
|  94819| 1396|
|  94851| 1372|
|  78113| 1362|
|  10817| 1355|
|  72820| 1353|
|   8389| 1338|
|  10914| 1366|
|   1522| 1332|
|  93633| 1356|
|  74440| 1374|
|  73041| 1360|
|  94621| 1326|
|  74452| 1344|
|  92858| 1392|
|  93131| 1371|
|    540| 1382|
| 100453| 1387|
|  68609| 1371|
|  90493| 1351|
|  78272| 1383|
|  95151| 1373|
|  76701| 1382|
|   2999| 1372|
|  89044| 1381|
|  88871| 1362|
|  94642| 1382|
|  74411| 1386|
|   5117| 1419|
|  71600| 1345|
|  11025| 1328|
|  88999| 1360|
| 102430| 1354|
|  92895| 1350|
|  97128| 1372|
|  93487| 1323|
|  72912| 1354|
|   8257| 1368|
|  86406| 1373|
|  73436| 1346|
|   8407| 1362|
|  92984| 1335|
|  97828| 1379|
|  94726| 1342|
|  66594| 1328|
|  78269| 1346|
|  91200| 1359|
|   9071| 1377|
|  74318| 1345|
|  94722| 1335|
|  72387| 1351|
|   7098| 1369|
|  81832

In [684]:
total_item_id_purchase = raw_train.groupBy("item_id").count().withColumnRenamed('count', 'total_item_id_purchase')

In [685]:
total_item_id_purchase.show()

+-------+----------------------+
|item_id|total_item_id_purchase|
+-------+----------------------+
|  74820|                  1344|
|  90019|                  1315|
|   8638|                  1379|
|  99817|                  1364|
|  93486|                  1401|
|  95080|                  1376|
|  74757|                  1359|
|  95940|                  1409|
|  94819|                  1396|
|  94851|                  1372|
|  78113|                  1362|
|  10817|                  1355|
|  72820|                  1353|
|   8389|                  1338|
|  10914|                  1366|
|   1522|                  1332|
|  93633|                  1356|
|  74440|                  1374|
|  73041|                  1360|
|  94621|                  1326|
+-------+----------------------+
only showing top 20 rows



In [686]:
total_item_id_purchase.count()

3704

In [687]:
raw_train.filter(raw_train.purchase == 1).groupBy("item_id").count().show(3704)

+-------+-----+
|item_id|count|
+-------+-----+
|  78113|    2|
|   8389|    8|
|  95940|    1|
|  94819|    1|
|  74820|    1|
|  90019|    3|
|  93486|    3|
|  72820|    1|
|   8638|    2|
|  74757|    1|
|  94851|    1|
|  10817|    1|
|  93633|    3|
|  74440|    5|
|    540|    1|
|   2999|    5|
|  74452|    2|
|  68609|    3|
|  93131|    1|
|  78272|    3|
|  73041|    1|
|  10914|    1|
|  95151|    2|
|  92858|    2|
|  76701|    1|
|   1522|    1|
|  94621|    1|
| 100453|    2|
|  92895|    1|
|   8257|   43|
|  74411|    1|
|  88999|    6|
|  97128|    2|
|  94642|    1|
|  88871|    1|
|  73436|    3|
| 102430|    1|
|   8407|    2|
|   5117|    2|
|  93487|    4|
|  92984|    1|
|  89044|    1|
|  66594|   21|
|  65879|    7|
| 100110|    1|
|  74318|    5|
|  93477|    1|
|   7644|    2|
|  91200|    1|
|  94726|    2|
|  97828|    1|
|  81832|    1|
|  88890|   10|
|  94768|    2|
|   5513|    1|
|  11482|    1|
|  87168|    7|
|  10788|    6|
|  73185|    3|
|   5682

In [688]:
item_id_purchase = raw_train.filter(raw_train.purchase == 1).groupBy("item_id").count().withColumnRenamed('count', 'item_id_purchase')

In [689]:
item_id_purchase.show()

+-------+----------------+
|item_id|item_id_purchase|
+-------+----------------+
|   8389|               8|
|  74757|               1|
|   8638|               2|
|  74820|               1|
|  90019|               3|
|  78113|               2|
|  93486|               3|
|  72820|               1|
|  94851|               1|
|  10817|               1|
|  95940|               1|
|  94819|               1|
|  74440|               5|
|  76701|               1|
|   2999|               5|
|   1522|               1|
|  94621|               1|
|  92858|               2|
|  93633|               3|
|  74452|               2|
+-------+----------------+
only showing top 20 rows



In [690]:
item_id_purchase.count()

3089

In [691]:
item_id_feature = total_item_id_purchase.join(item_id_purchase, on="item_id", how="outer")

In [692]:
item_id_feature.show()

+-------+----------------------+----------------+
|item_id|total_item_id_purchase|item_id_purchase|
+-------+----------------------+----------------+
|   8389|                  1338|               8|
|   8638|                  1379|               2|
|  10817|                  1355|               1|
|  72820|                  1353|               1|
|  74757|                  1359|               1|
|  74820|                  1344|               1|
|  78113|                  1362|               2|
|  90019|                  1315|               3|
|  93486|                  1401|               3|
|  94819|                  1396|               1|
|  94851|                  1372|               1|
|  95080|                  1376|            null|
|  95940|                  1409|               1|
|  99817|                  1364|            null|
|    540|                  1382|               1|
|   1522|                  1332|               1|
|   2999|                  1372|               5|


In [693]:
item_id_feature.count()

3704

In [694]:
active_item_purchase = item_id_feature.select("item_id", del_purchase("item_id_purchase", "total_item_id_purchase").alias("active_item_purchase"))

In [695]:
active_item_purchase.show()

+-------+--------------------+
|item_id|active_item_purchase|
+-------+--------------------+
|   8389|        0.0059790732|
|   8638|        0.0014503263|
|  10817|        7.3800737E-4|
|  72820|         7.390983E-4|
|  74757|         7.358352E-4|
|  74820|        7.4404763E-4|
|  78113|        0.0014684288|
|  90019|        0.0022813687|
|  93486|        0.0021413276|
|  94819|         7.163324E-4|
|  94851|          7.28863E-4|
|  95080|                null|
|  95940|         7.097232E-4|
|  99817|                null|
|    540|          7.23589E-4|
|   1522|        7.5075077E-4|
|   2999|        0.0036443148|
|  10914|         7.320644E-4|
|  68609|        0.0021881838|
|  73041|        7.3529413E-4|
+-------+--------------------+
only showing top 20 rows



In [696]:
active_item_purchase.count()

3704

In [697]:
active_item_purchase.filter(f.isnull("active_item_purchase")).count()

615

In [698]:
df_train.show()

+-------+-------+--------+--------------------+
|user_id|item_id|purchase|active_user_purchase|
+-------+-------+--------+--------------------+
| 754230|  73019|       0|         0.027575642|
| 754230|  10558|       0|         0.027575642|
| 754230|  74581|       0|         0.027575642|
| 754230|  74434|       0|         0.027575642|
| 754230|  92356|       0|         0.027575642|
| 754230|  72931|       0|         0.027575642|
| 754230|  98246|       0|         0.027575642|
| 754230|  88927|       0|         0.027575642|
| 754230|  10643|       1|         0.027575642|
| 754230|  10593|       0|         0.027575642|
| 754230|  93153|       0|         0.027575642|
| 754230|  75384|       0|         0.027575642|
| 754230|  88996|       0|         0.027575642|
| 754230|  73411|       0|         0.027575642|
| 754230|  72769|       0|         0.027575642|
| 754230|   5193|       0|         0.027575642|
| 754230|  72845|       0|         0.027575642|
| 754230| 100246|       0|         0.027

In [699]:
df_train = df_train.join(active_item_purchase, on="item_id", how="outer")

In [700]:
df_train.show()

+-------+-------+--------+--------------------+--------------------+
|item_id|user_id|purchase|active_user_purchase|active_item_purchase|
+-------+-------+--------+--------------------+--------------------+
|   8389| 754230|       0|         0.027575642|        0.0059790732|
|   8389| 780033|       0|         7.757952E-4|        0.0059790732|
|   8389| 798454|       0|        3.8402458E-4|        0.0059790732|
|   8389| 825061|       0|        0.0019312475|        0.0059790732|
|   8389| 833685|       0|         0.007500987|        0.0059790732|
|   8389| 851486|       0|                null|        0.0059790732|
|   8389| 867850|       0|        3.8299503E-4|        0.0059790732|
|   8389| 870928|       0|         7.674597E-4|        0.0059790732|
|   8389| 879401|       0|         0.004283489|        0.0059790732|
|   8389| 901457|       0|                null|        0.0059790732|
|   8389| 927211|       0|        3.9169606E-4|        0.0059790732|
|   8389| 928140|       0|        

In [701]:
df_train.filter(df_train.item_id == 2999).show()

+-------+-------+--------+--------------------+--------------------+
|item_id|user_id|purchase|active_user_purchase|active_item_purchase|
+-------+-------+--------+--------------------+--------------------+
|   2999| 776138|       0|                0.01|        0.0036443148|
|   2999| 806704|       0|        0.0011560693|        0.0036443148|
|   2999| 833838|       0|         0.005740528|        0.0036443148|
|   2999| 865948|       0|        3.8211694E-4|        0.0036443148|
|   2999| 868207|       0|        3.9277298E-4|        0.0036443148|
|   2999| 876758|       0|        0.0011709601|        0.0036443148|
|   2999| 896115|       0|        0.0042242706|        0.0036443148|
|   2999| 902417|       0|        3.8461538E-4|        0.0036443148|
|   2999| 907510|       0|                null|        0.0036443148|
|   2999| 936094|       0|         0.004265219|        0.0036443148|
|   2999| 859061|       0|        7.7041605E-4|        0.0036443148|
|   2999| 866647|       0|        

In [702]:
df_train.show()

+-------+-------+--------+--------------------+--------------------+
|item_id|user_id|purchase|active_user_purchase|active_item_purchase|
+-------+-------+--------+--------------------+--------------------+
|   8389| 754230|       0|         0.027575642|        0.0059790732|
|   8389| 780033|       0|         7.757952E-4|        0.0059790732|
|   8389| 798454|       0|        3.8402458E-4|        0.0059790732|
|   8389| 825061|       0|        0.0019312475|        0.0059790732|
|   8389| 833685|       0|         0.007500987|        0.0059790732|
|   8389| 851486|       0|                null|        0.0059790732|
|   8389| 867850|       0|        3.8299503E-4|        0.0059790732|
|   8389| 870928|       0|         7.674597E-4|        0.0059790732|
|   8389| 879401|       0|         0.004283489|        0.0059790732|
|   8389| 901457|       0|                null|        0.0059790732|
|   8389| 927211|       0|        3.9169606E-4|        0.0059790732|
|   8389| 928140|       0|        

In [703]:
df_train.count()

5032624

In [75]:
df_train.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- purchase: integer (nullable = true)
 |-- active_user_purchase: float (nullable = true)
 |-- active_item_purchase: float (nullable = true)



# raw_test

In [704]:
schema_test = StructType([
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType())
])

In [705]:
raw_test = spark.read.csv("/labs/slaba03/laba03_test.csv", sep=",", schema=schema_test, header=True)

In [706]:
raw_test.rdd.getNumPartitions()

5

In [707]:
raw_test = raw_test.repartition(10)

In [708]:
raw_test.rdd.getNumPartitions()

10

In [709]:
raw_test.show()

+-------+-------+
|user_id|item_id|
+-------+-------+
| 779657|   5340|
| 775974|  76255|
| 588378|   9956|
| 834532|  10488|
| 760629|  10234|
| 833424| 100489|
| 799703|  66447|
| 732411|  10855|
| 522798|  10209|
| 813275|  86406|
| 788108| 100172|
| 799180| 102431|
| 811851|  95015|
| 746365|  92802|
| 840226|   4283|
| 816738|  81780|
| 791395|  80892|
| 814802|  73487|
| 799148|  86846|
| 826271|  92532|
+-------+-------+
only showing top 20 rows



In [710]:
raw_test.select('user_id').distinct().count()

1941

In [711]:
user_id_list = raw_test.select('user_id').distinct().collect()

test_user_id_list = []
[test_user_id_list.append(user_id_list[i][0]) for i in range(len(user_id_list))]
test_user_id_list = set(test_user_id_list)

train_user_id_list == test_user_id_list

True

In [712]:
raw_test.select('item_id').distinct().count()

3704

In [713]:
user_item_list = raw_test.select('item_id').distinct().collect()

test_item_id_list = []
[test_item_id_list.append(item_id_list[i][0]) for i in range(len(item_id_list))]
test_item_id_list = set(test_item_id_list)

train_item_id_list == test_item_id_list

True

In [714]:
df_train.show()

+-------+-------+--------+--------------------+--------------------+
|item_id|user_id|purchase|active_user_purchase|active_item_purchase|
+-------+-------+--------+--------------------+--------------------+
|   8389| 746713|       0|                null|        0.0059790732|
|   8389| 883098|       0|         0.001948558|        0.0059790732|
|   8389| 903491|       0|        0.0011614402|        0.0059790732|
|   8389| 903826|       0|        0.0015444015|        0.0059790732|
|   8389| 916566|       0|        3.8402458E-4|        0.0059790732|
|   8389| 819462|       0|         3.844675E-4|        0.0059790732|
|   8389| 834532|       0|        0.0011450382|        0.0059790732|
|   8389| 876234|       0|                null|        0.0059790732|
|   8389| 913290|       0|         3.900156E-4|        0.0059790732|
|   8389| 928490|       0|        0.0015390535|        0.0059790732|
|   8389| 842784|       0|                null|        0.0059790732|
|   8389| 861601|       0|        

In [715]:
df_test = raw_test.join(active_user_purchase, on="user_id", how="outer")

In [716]:
df_test.show()

+-------+-------+--------------------+
|user_id|item_id|active_user_purchase|
+-------+-------+--------------------+
| 754230|  94910|         0.027575642|
| 754230| 102033|         0.027575642|
| 754230|  93732|         0.027575642|
| 754230|  79890|         0.027575642|
| 754230|  77799|         0.027575642|
| 754230| 103589|         0.027575642|
| 754230|  73274|         0.027575642|
| 754230|  93488|         0.027575642|
| 754230|  74626|         0.027575642|
| 754230|  81824|         0.027575642|
| 754230|  92457|         0.027575642|
| 754230|  94637|         0.027575642|
| 754230|  93575|         0.027575642|
| 754230|  10825|         0.027575642|
| 754230|  98984|         0.027575642|
| 754230|  72329|         0.027575642|
| 754230|  94614|         0.027575642|
| 754230| 103580|         0.027575642|
| 754230|  94819|         0.027575642|
| 754230|  10807|         0.027575642|
+-------+-------+--------------------+
only showing top 20 rows



In [717]:
df_test = df_test.join(active_item_purchase, on="item_id", how="outer")

In [718]:
df_test.show()

+-------+-------+--------------------+--------------------+
|item_id|user_id|active_user_purchase|active_item_purchase|
+-------+-------+--------------------+--------------------+
|   8389| 761341|         3.875969E-4|        0.0059790732|
|   8389| 776188|        0.0011525163|        0.0059790732|
|   8389| 846231|        0.0019238169|        0.0059790732|
|   8389| 822709|         3.789314E-4|        0.0059790732|
|   8389| 824008|        3.8211694E-4|        0.0059790732|
|   8389| 890476|                null|        0.0059790732|
|   8389| 899993|        3.8895372E-4|        0.0059790732|
|   8389| 937345|        3.8505968E-4|        0.0059790732|
|   8389| 566758|        3.7664783E-4|        0.0059790732|
|   8389| 816426|                null|        0.0059790732|
|   8389| 892290|        7.7190273E-4|        0.0059790732|
|   8389| 740836|        0.0027788805|        0.0059790732|
|   8389| 866581|        0.0015917231|        0.0059790732|
|   8389| 901042|        3.8138826E-4|  

# raw_items

In [535]:
!hdfs dfs -head /labs/slaba03/laba03_items.csv

item_id	channel_id	datetime_availability_start	datetime_availability_stop	datetime_show_start	datetime_show_stop	content_type	title	year	genres	region_id
65667		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			1	на пробах только девушки (all girl auditions)	2013.0	Эротика	
65669		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			1	скуби ду: эротическая пародия (scooby doo: a xxx parody)	2011.0	Эротика	
65668		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			1	горячие девочки для горячих девочек (hot babes 4 hot babes)	2011.0	Эротика	
65671		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			1	соблазнительницы женатых мужчин (top heavy homewreckers)	2011.0	Эротика	
65670		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			1	секретные секс-материалы ii: темная секс пародия (the sex files ii: a dark xxx parody)	2010.0	Эротика	
65809		1970-01-01T00:00:00Z	2099-12-31

In [536]:
raw_items = spark.read.csv("/labs/slaba03/laba03_items.csv", sep="\t", header=True, multiLine=True)

In [537]:
raw_items.show(3, truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------
 item_id                     | 65667                                                       
 channel_id                  | null                                                        
 datetime_availability_start | 1970-01-01T00:00:00Z                                        
 datetime_availability_stop  | 2018-01-01T00:00:00Z                                        
 datetime_show_start         | null                                                        
 datetime_show_stop          | null                                                        
 content_type                | 1                                                           
 title                       | на пробах только девушки (all girl auditions)               
 year                        | 2013.0                                                      
 genres                      | Эротика                                          

In [538]:
raw_items.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- channel_id: string (nullable = true)
 |-- datetime_availability_start: string (nullable = true)
 |-- datetime_availability_stop: string (nullable = true)
 |-- datetime_show_start: string (nullable = true)
 |-- datetime_show_stop: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- region_id: string (nullable = true)



In [539]:
schema_items = StructType([
    StructField("item_id", IntegerType()),
    StructField("channel_id", IntegerType()),
    StructField("datetime_availability_start", StringType()),
    StructField ("datetime_availability_stop", StringType()),
    StructField("datetime_show_start", StringType()),
    StructField("datetime_show_stop", StringType()),
    StructField("content_type", IntegerType()),
    StructField("title", StringType()),
    StructField("year", FloatType()),
    StructField("genres",StringType()),
    StructField("region_id", IntegerType())
])

In [540]:
raw_items = spark.read.csv("/labs/slaba03/laba03_items.csv", sep="\t", schema=schema_items, header=True, multiLine=True, escape='"')

In [541]:
raw_items.rdd.getNumPartitions()

1

In [542]:
raw_items = raw_items.repartition(10)

In [543]:
raw_items.rdd.getNumPartitions()

10

In [544]:
raw_items.show(1, truncate=False, vertical=True)

-RECORD 0-------------------------------------------
 item_id                     | 11099                
 channel_id                  | null                 
 datetime_availability_start | 1970-01-01T00:00:00Z 
 datetime_availability_stop  | 2099-12-31T21:00:00Z 
 datetime_show_start         | null                 
 datetime_show_stop          | null                 
 content_type                | 1                    
 title                       | сваты 4              
 year                        | 2010.0               
 genres                      | Комедии,Наши         
 region_id                   | null                 
only showing top 1 row



In [545]:
raw_items.count()

635568

In [546]:
import pyspark.sql.functions as f

In [547]:
raw_items.filter(f.isnull("channel_id")).count()

635568

In [548]:
raw_items.select("datetime_availability_start", "datetime_availability_stop").show()

+---------------------------+--------------------------+
|datetime_availability_start|datetime_availability_stop|
+---------------------------+--------------------------+
|       1970-01-01T00:00:00Z|      2017-04-07T12:45:00Z|
|       1970-01-01T00:00:00Z|      2017-05-06T10:45:00Z|
|       1970-01-01T00:00:00Z|      2017-03-26T05:25:00Z|
|       1970-01-01T00:00:00Z|      2017-04-06T22:05:00Z|
|       1970-01-01T00:00:00Z|      2017-04-06T22:05:00Z|
|       1970-01-01T00:00:00Z|      2017-03-03T05:10:00Z|
|       1970-01-01T00:00:00Z|      2017-03-03T05:10:00Z|
|       1970-01-01T00:00:00Z|      2017-02-16T08:45:00Z|
|       1970-01-01T00:00:00Z|      2017-04-10T22:45:00Z|
|       1970-01-01T00:00:00Z|      2017-04-23T07:30:00Z|
|       1970-01-01T00:00:00Z|      2017-04-23T07:30:00Z|
|       1970-01-01T00:00:00Z|      2017-04-23T07:30:00Z|
|       1970-01-01T00:00:00Z|      2017-04-23T07:30:00Z|
|       1970-01-01T00:00:00Z|      2017-04-23T07:30:00Z|
|       1970-01-01T00:00:00Z|  

In [549]:
raw_items.filter(f.isnull("datetime_show_start")).count()

3704

In [550]:
raw_items.filter(f.isnull("datetime_show_stop")).count()

3704

In [551]:
raw_items.select("datetime_show_start", "datetime_show_stop").show(5000)

+--------------------+--------------------+
| datetime_show_start|  datetime_show_stop|
+--------------------+--------------------+
|2017-04-07T04:15:00Z|2017-04-07T05:35:00Z|
|2017-04-03T09:15:00Z|2017-04-03T10:20:00Z|
|2017-02-16T00:30:00Z|2017-02-16T02:00:00Z|
|2017-04-01T05:35:00Z|2017-04-01T06:00:00Z|
|2017-04-23T03:45:00Z|2017-04-23T04:00:00Z|
|2017-04-19T20:10:00Z|2017-04-19T21:00:00Z|
|2017-04-19T20:10:00Z|2017-04-19T21:00:00Z|
|2017-04-14T16:55:00Z|2017-04-14T17:20:00Z|
|2017-03-14T15:55:00Z|2017-03-14T16:55:00Z|
|2017-02-27T21:35:00Z|2017-02-27T22:00:00Z|
|2017-02-13T11:00:00Z|2017-02-13T11:35:00Z|
|2017-02-15T00:15:00Z|2017-02-15T00:55:00Z|
|2017-04-20T06:00:00Z|2017-04-20T08:00:00Z|
|2017-03-08T08:45:00Z|2017-03-08T09:00:00Z|
|2017-03-14T21:10:00Z|2017-03-14T21:35:00Z|
|2017-02-27T22:45:00Z|2017-02-27T23:10:00Z|
|2017-04-28T01:45:00Z|2017-04-28T02:00:00Z|
|2017-02-14T06:15:00Z|2017-02-14T06:25:00Z|
|2017-03-17T18:20:00Z|2017-03-17T18:45:00Z|
|2017-04-16T17:10:00Z|2017-04-16

In [552]:
raw_items.filter(f.isnull("region_id")).count()

635568

In [553]:
raw_items = raw_items.drop("channel_id",
                           "datetime_availability_start",
                           "datetime_availability_stop",
                           "datetime_show_start",
                           "datetime_show_stop",
                           "region_id")

In [554]:
raw_items.show(1, truncate=False, vertical=True)

-RECORD 0---------------
 item_id      | 6320010 
 content_type | 0       
 title        | события 
 year         | null    
 genres       | General 
only showing top 1 row



In [555]:
raw_items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- content_type: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- year: float (nullable = true)
 |-- genres: string (nullable = true)



In [556]:
raw_items.show(10000)

+-------+------------+--------------------+------+--------------------+
|item_id|content_type|               title|  year|              genres|
+-------+------------+--------------------+------+--------------------+
|6320010|           0|             события|  null|             General|
|6363901|           0|дикая франция с р...|  null|             General|
|6947414|           0|       время малышей|  null|             General|
|7067658|           0|   белорусская кухня|  null|             General|
|6296176|           0|        петросян-шоу|  null|             General|
|6964259|           0|специалист по сва...|  null|             General|
|7006131|           0|            6 кадров|  null|             General|
|7439390|           0|      автомастерские|  null|             General|
|6627270|           0|            наше все|  null|             General|
|6969249|           0|    сделано в россии|  null|             General|
|7339433|           0|     острый репортаж|  null|             G

In [557]:
raw_items.filter(raw_items.content_type == 1).show(10000)

+-------+------------+--------------------+------+--------------------+
|item_id|content_type|               title|  year|              genres|
+-------+------------+--------------------+------+--------------------+
|  92542|           1|        хроники тьмы|2009.0|Приключения,Фэнте...|
|  84449|           1|     друзья навсегда|2009.0|Про животных,Полн...|
|  10914|           1|             фазенда|2012.0|      Познавательные|
|  99743|           1|              бывшая|2007.0|Романтические,Зар...|
|  73215|           1|   танька и володька|2016.0|             Комедии|
|  78026|           1|          малышарики|2015.0|Русские мультфиль...|
|  94734|           1|     кто сказал мяу?|1962.0|Русские мультфиль...|
|  66577|           1|война богов: бесс...|2011.0|Драмы,Фэнтези,Бое...|
|  10556|           1|самые странные бо...|2015.0|      Документальные|
|  92151|           1|            топтыжка|1964.0|Про животных,Русс...|
|   8658|           1|             фиксики|2010.0|Мультфильмы,Му

In [558]:
raw_items = raw_items.filter(raw_items.content_type == 1)

In [559]:
raw_items.show(10000)

+-------+------------+--------------------+------+--------------------+
|item_id|content_type|               title|  year|              genres|
+-------+------------+--------------------+------+--------------------+
|  92542|           1|        хроники тьмы|2009.0|Приключения,Фэнте...|
|  84449|           1|     друзья навсегда|2009.0|Про животных,Полн...|
|  10914|           1|             фазенда|2012.0|      Познавательные|
|  99743|           1|              бывшая|2007.0|Романтические,Зар...|
|  73215|           1|   танька и володька|2016.0|             Комедии|
|  78026|           1|          малышарики|2015.0|Русские мультфиль...|
|  94734|           1|     кто сказал мяу?|1962.0|Русские мультфиль...|
|  66577|           1|война богов: бесс...|2011.0|Драмы,Фэнтези,Бое...|
|  10556|           1|самые странные бо...|2015.0|      Документальные|
|  92151|           1|            топтыжка|1964.0|Про животных,Русс...|
|   8658|           1|             фиксики|2010.0|Мультфильмы,Му

In [560]:
raw_items.count()

3704

In [277]:
# user_item_list = raw_items.select('item_id').distinct().collect()

# raw_items_item_id_list = []
# [raw_items_item_id_list.append(user_item_list[i][0]) for i in range(len(user_item_list))]
# raw_items_item_id_list = set(raw_items_item_id_list)

# train_item_id_list == test_item_id_list == raw_items_item_id_list

True

In [561]:
raw_items = raw_items.drop("content_type")

In [562]:
raw_items.filter(f.isnull("title")).count()

0

In [563]:
raw_items.filter(f.isnull("year")).count()

4

In [564]:
raw_items.filter(f.isnull("year")).show(4)

+-------+--------------------+----+--------------------+
|item_id|               title|year|              genres|
+-------+--------------------+----+--------------------+
| 103377|big buck bunny 1080p|null|                null|
|   8544|                30-е|null|              Прочие|
|  72544|     городские герои|null|Для всей семьи,Дл...|
|  95141|   современный роман|null|             Эротика|
+-------+--------------------+----+--------------------+



In [565]:
raw_items.filter(f.isnull("genres")).count()

33

In [566]:
raw_items.filter(f.isnull("genres")).show(33)

+-------+--------------------+------+------+
|item_id|               title|  year|genres|
+-------+--------------------+------+------+
|  94979|знает кошка, чью ...|2015.0|  null|
|   9717| больница никербокер|2014.0|  null|
| 100247|     карточный домик|2013.0|  null|
|   9819|               звери|2016.0|  null|
|   9668|  покажите мне героя|2015.0|  null|
|  94988|новичкам везет (ч...|2015.0|  null|
|   9896|   девушка по вызову|2016.0|  null|
| 103377|big buck bunny 1080p|  null|  null|
|  94975| гламурный хардкор 3|2015.0|  null|
|  94974|    душевные страсти|2016.0|  null|
|   9898|             империя|2015.0|  null|
|   9817|         оставленные|2014.0|  null|
|   9180|      игра престолов|2011.0|  null|
|   9059|               банши|2013.0|  null|
|   9055|американская исто...|2011.0|  null|
|   6151|              родина|2011.0|  null|
|   9667|         рэй донован|2013.0|  null|
|  94978|      сладкая помада|2016.0|  null|
|   9914|        клан сопрано|1999.0|  null|
|   9897| 

In [567]:
item_id_null_list = [8544, 72544, 95141, 94979, 9717, 100247, 9819, 9668, 94988, 9896, 103377, 94975, 94974, 9898, 9817, 9180, 9059, 9055, 6151, 9667, 94978, 9914, 9897, 9720, 94977, 94973, 9064, 100248, 9632, 9062, 10205, 10208, 9721, 9699, 9821, 94976]
len(item_id_null_list)

36

In [ ]:
raw_train.filter(raw_train.purchase == 1).groupBy("item_id").count().filter(raw_train.item_id == 9180).show()

In [569]:
raw_items.count()

3704

In [570]:
raw_items.filter(f.isnull("year") | f.isnull("genres")).count()

36

In [571]:
raw_items.filter(f.isnull("year") | f.isnull("genres")).show(36)

+-------+--------------------+------+--------------------+
|item_id|               title|  year|              genres|
+-------+--------------------+------+--------------------+
|   9699|               винил|2016.0|                null|
|   9821|       плоть и кости|2015.0|                null|
|   9721| карточный домик old|2013.0|                null|
|   9819|               звери|2016.0|                null|
|  94979|знает кошка, чью ...|2015.0|                null|
|   9717| больница никербокер|2014.0|                null|
|  95141|   современный роман|  null|             Эротика|
| 103377|big buck bunny 1080p|  null|                null|
|   9668|  покажите мне героя|2015.0|                null|
|  94988|новичкам везет (ч...|2015.0|                null|
|  94976|    лисы в курятнике|2016.0|                null|
|   9817|         оставленные|2014.0|                null|
|  94975| гламурный хардкор 3|2015.0|                null|
|  94974|    душевные страсти|2016.0|                nul

In [572]:
raw_items.filter(~ (f.isnull("year") | f.isnull("genres"))).count()

3668

In [573]:
raw_items_3668 = raw_items.filter(~ (f.isnull("year") | f.isnull("genres")))

In [574]:
raw_items_3668.count()

3668

In [575]:
raw_items_3668.show()

+-------+--------------------+------+--------------------+
|item_id|               title|  year|              genres|
+-------+--------------------+------+--------------------+
|  89008|машина времени в ...|2010.0|Комедии,Фантастик...|
|  72849|освобождение: огн...|1968.0|Военные,Драмы,Мел...|
|  74106|              турист|2011.0|Ужасы,Триллеры,За...|
|  93620|фархат: принц пер...|2009.0|Мультфильмы,Мульт...|
|  77690|             бригада|2002.0|Драмы,Русские,Бое...|
|  95724|медсестры на задн...|2012.0|  Эротика,Зарубежные|
|  72605|    волшебники двора|2015.0|Русские мультфиль...|
|  69574|            спасение|2014.0|Драмы,Вестерн,Зар...|
|  65908|         три девятки|2016.0|Драмы,Криминал,За...|
|  92916|           заложники|2008.0|Русские,Документа...|
|  72894|где находится ноф...|1987.0|Комедии,Мелодрамы...|
|   9855|           ускорение|2015.0|Триллеры,Боевики,...|
|  83573| тимур и его команда|1940.0| Приключения,Боевики|
|   2736|       приговоренные|2007.0|Триллеры,Боевики,..

In [576]:
raw_items_3668 = raw_items_3668.select("item_id", "title", "year", "genres", f.split("genres", ",").alias("genres_list"))

In [577]:
raw_items_3668.show()

+-------+--------------------+------+--------------------+--------------------+
|item_id|               title|  year|              genres|         genres_list|
+-------+--------------------+------+--------------------+--------------------+
|  89008|машина времени в ...|2010.0|Комедии,Фантастик...|[Комедии, Фантаст...|
|  72849|освобождение: огн...|1968.0|Военные,Драмы,Мел...|[Военные, Драмы, ...|
|  74106|              турист|2011.0|Ужасы,Триллеры,За...|[Ужасы, Триллеры,...|
|  93620|фархат: принц пер...|2009.0|Мультфильмы,Мульт...|[Мультфильмы, Мул...|
|  77690|             бригада|2002.0|Драмы,Русские,Бое...|[Драмы, Русские, ...|
|  95724|медсестры на задн...|2012.0|  Эротика,Зарубежные|[Эротика, Зарубеж...|
|  72605|    волшебники двора|2015.0|Русские мультфиль...|[Русские мультфил...|
|  69574|            спасение|2014.0|Драмы,Вестерн,Зар...|[Драмы, Вестерн, ...|
|  65908|         три девятки|2016.0|Драмы,Криминал,За...|[Драмы, Криминал,...|
|  92916|           заложники|2008.0|Рус

In [578]:
raw_items_3668.select(f.explode("genres_list").alias("unic_genres")).groupby("unic_genres").count().orderBy("unic_genres", ascending=True).show(83)

+--------------------+-----+
|         unic_genres|count|
+--------------------+-----+
|              сказка|    3|
|            Анимация|    2|
|               Аниме|    8|
|            Арт-хаус|    6|
|             Артхаус|   26|
|           Биография|   44|
|              Боевик|   12|
|             Боевики|  543|
|             Вестерн|   11|
|           Видеоигры|   22|
|             Военные|  175|
|             Военный|    4|
|           Детективы|  355|
|             Детские|  131|
|       Детские песни|    7|
|        Для взрослых|   45|
|      Для всей семьи|  221|
|           Для детей|  427|
| Для самых маленьких|   19|
|      Документальные|  103|
|      Документальный|    6|
|               Драма|   34|
|               Драмы|  957|
|Западные мультфильмы|  118|
|          Зарубежные| 1739|
|                Игры|    8|
|        Исторические|  116|
|        Исторический|    9|
|             Комедии|  857|
|             Комедия|   18|
|     Короткометражки|    3|
|    Короткоме

In [579]:
row_items_36 = [(8544, "30-е", 2015.0, "Ужасы,Триллеры"),\
                (72544, "городские герои", 2009.0, "Для всей семьи,Для детей,Зарубежные"),\
                (95141, "современный роман", 2015.0, "Эротика"),\
                (94979, "знает кошка, чью сметану съела", 2015.0, "Прочие"),\
                (9717, "больница никербокер", 2014.0, "Сериалы,Драмы,Исторические"),\
                (100247, "карточный домик", 2013.0, "Сериалы,Драмы"),\
                (9819, "звери", 2016.0, "Детские,Мультфильмы"),\
                (9668, "покажите мне героя", 2015.0, "Сериалы,Драмы,Исторические"),\
                (94988, "новичкам везет (часть 2)", 2015.0, "Эротика"),\
                (9896, "девушка по вызову", 2016.0, "Драмы"),\
                (103377,"big buck bunny 1080p", 2008.0, "Детские,Мультфильмы"),\
                (94975,"гламурный хардкор 3", 2015.0, "Эротика"),\
                (94974, "душевные страсти", 2016.0, "Прочие"),\
                (9898, "империя", 2015.0, "Сериалы,Драмы,Музыкальные"),\
                (9817, "оставленные", 2014.0, "Сериалы,Драмы,Детективы,Фэнтези"),\
                (9180, "игра престолов", 2011.0, "Сериалы,Фэнтези,Драмы,Боевики,Приключения,Мелодрамы"),\
                (9059, "банши", 2013.0, "Прочие"),\
                (9055, "американская история ужасов old", 2011.0,"Ужасы,Фантастика,Драмы,Триллеры"),\
                (6151, "родина", 2011.0, "Сериалы,Драмы,Триллеры,Криминал,Детективы"),\
                (9667, "рэй донован", 2013.0, "Сериалы,Драмы,Криминал"),\
                (94978, "сладкая помада", 2016.0, "Эротика"),\
                (9914, "клан сопрано", 1999.0, "Сериалы,Драмы,Криминал"),\
                (9897, "вместе", 2015.0, "Сериалы,Драмы,Мелодрамы,Комедии"),\
                (9720, "черные паруса", 2014.0, "Сериалы,Драмы,Боевики,Приключения"),\
                (94977, "новичкам везет (часть 1)", 2015.0, "Эротика"),\
                (94973, "бруклин ли. дневник нимфоманки", 2016.0, "Эротика"),\
                (9064, "настоящий детектив", 2014.0, "Сериалы,Драмы,Триллеры,Криминал,Детективы"),\
                (100248, "американская история ужасов", 2011.0, "Ужасы,Фантастика,Драмы,Триллеры"),\
                (9632, "силиконовая долина", 2014.0, "Сериалы,Комедии"),\
                (9062, "вице-президент", 2012.0, "Сериалы,Комедии"),\
                (10205, "метод", 2015.0, "Сериалы,Драмы,Триллеры,Криминал"),\
                (10208, "мост", 2011.0, "Сериалы,Триллеры,Криминал,Детективы"),\
                (9721, "карточный домик old", 2013.0, "Сериалы,Драмы"),\
                (9699, "винил", 2016.0, "Сериалы,Драмы,Музыкальные"),\
                (9821, "плоть и кости", 2015.0, "Сериалы,Драмы"),\
                (94976, "лисы в курятнике", 2016.0, "Прочие")]

In [580]:
schema_items_36 = StructType([
    StructField("item_id", IntegerType()),
    StructField("title", StringType()),
    StructField("year", FloatType()),
    StructField("genres",StringType()),
])

In [581]:
row_items_36 = spark.createDataFrame(row_items_36, schema=schema_items_36)

In [582]:
row_items_36.show(36)

+-------+--------------------+------+--------------------+
|item_id|               title|  year|              genres|
+-------+--------------------+------+--------------------+
|   8544|                30-е|2015.0|      Ужасы,Триллеры|
|  72544|     городские герои|2009.0|Для всей семьи,Дл...|
|  95141|   современный роман|2015.0|             Эротика|
|  94979|знает кошка, чью ...|2015.0|              Прочие|
|   9717| больница никербокер|2014.0|Сериалы,Драмы,Ист...|
| 100247|     карточный домик|2013.0|       Сериалы,Драмы|
|   9819|               звери|2016.0| Детские,Мультфильмы|
|   9668|  покажите мне героя|2015.0|Сериалы,Драмы,Ист...|
|  94988|новичкам везет (ч...|2015.0|             Эротика|
|   9896|   девушка по вызову|2016.0|               Драмы|
| 103377|big buck bunny 1080p|2008.0| Детские,Мультфильмы|
|  94975| гламурный хардкор 3|2015.0|             Эротика|
|  94974|    душевные страсти|2016.0|              Прочие|
|   9898|             империя|2015.0|Сериалы,Драмы,Муз..

In [583]:
row_items_36.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- year: float (nullable = true)
 |-- genres: string (nullable = true)



In [262]:
row_items_36.count()

36

In [584]:
row_items_36 = row_items_36.select("item_id", "title", "year", "genres", f.split("genres", ",").alias("genres_list"))

In [585]:
row_items_36.show()

+-------+--------------------+------+--------------------+--------------------+
|item_id|               title|  year|              genres|         genres_list|
+-------+--------------------+------+--------------------+--------------------+
|   8544|                30-е|2015.0|      Ужасы,Триллеры|   [Ужасы, Триллеры]|
|  72544|     городские герои|2009.0|Для всей семьи,Дл...|[Для всей семьи, ...|
|  95141|   современный роман|2015.0|             Эротика|           [Эротика]|
|  94979|знает кошка, чью ...|2015.0|              Прочие|            [Прочие]|
|   9717| больница никербокер|2014.0|Сериалы,Драмы,Ист...|[Сериалы, Драмы, ...|
| 100247|     карточный домик|2013.0|       Сериалы,Драмы|    [Сериалы, Драмы]|
|   9819|               звери|2016.0| Детские,Мультфильмы|[Детские, Мультфи...|
|   9668|  покажите мне героя|2015.0|Сериалы,Драмы,Ист...|[Сериалы, Драмы, ...|
|  94988|новичкам везет (ч...|2015.0|             Эротика|           [Эротика]|
|   9896|   девушка по вызову|2016.0|   

In [586]:
row_items_36.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- year: float (nullable = true)
 |-- genres: string (nullable = true)
 |-- genres_list: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [267]:
# from pyspark.sql import SparkSession
# import functools

# def unionAll(dfs):
#     return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

In [268]:
# unioned_raw_items = unionAll([raw_items_3668, row_items_36])

In [587]:
unioned_raw_items = raw_items_3668.union(row_items_36)

In [278]:
# unioned_raw_items.filter("title LIKE '%old%'").show(5, truncate=False, vertical=True)

-RECORD 0-------------------------------------------
 item_id     | 9055                                 
 title       | американская история ужасов old      
 year        | 2011.0                               
 genres      | Ужасы,Фантастика,Драмы,Триллеры      
 genres_list | [Ужасы, Фантастика, Драмы, Триллеры] 
 title_list  | [американская, история, ужасов, old] 
-RECORD 1-------------------------------------------
 item_id     | 9721                                 
 title       | карточный домик old                  
 year        | 2013.0                               
 genres      | Сериалы,Драмы                        
 genres_list | [Сериалы, Драмы]                     
 title_list  | [карточный, домик, old]              



In [588]:
unioned_raw_items.show()

+-------+--------------------+------+--------------------+--------------------+
|item_id|               title|  year|              genres|         genres_list|
+-------+--------------------+------+--------------------+--------------------+
|  89008|машина времени в ...|2010.0|Комедии,Фантастик...|[Комедии, Фантаст...|
|  72849|освобождение: огн...|1968.0|Военные,Драмы,Мел...|[Военные, Драмы, ...|
|  74106|              турист|2011.0|Ужасы,Триллеры,За...|[Ужасы, Триллеры,...|
|  93620|фархат: принц пер...|2009.0|Мультфильмы,Мульт...|[Мультфильмы, Мул...|
|  77690|             бригада|2002.0|Драмы,Русские,Бое...|[Драмы, Русские, ...|
|  95724|медсестры на задн...|2012.0|  Эротика,Зарубежные|[Эротика, Зарубеж...|
|  72605|    волшебники двора|2015.0|Русские мультфиль...|[Русские мультфил...|
|  69574|            спасение|2014.0|Драмы,Вестерн,Зар...|[Драмы, Вестерн, ...|
|  65908|         три девятки|2016.0|Драмы,Криминал,За...|[Драмы, Криминал,...|
|  92916|           заложники|2008.0|Рус

In [589]:
unioned_raw_items.count()

3704

In [590]:
unioned_raw_items.filter(f.isnull("year")).count()

0

In [591]:
unioned_raw_items.filter(f.isnull("genres")).count()

0

In [592]:
unioned_raw_items.filter(f.isnull("genres_list")).count()

0

In [601]:
from pyspark.ml.feature import HashingTF, CountVectorizer, IDF, Tokenizer, StopWordsRemover

In [594]:
tokenizer_ru = Tokenizer(inputCol="title", outputCol="title_list")
unioned_raw_items = tokenizer_ru.transform(unioned_raw_items)

In [595]:
unioned_raw_items.show()

+-------+--------------------+------+--------------------+--------------------+--------------------+
|item_id|               title|  year|              genres|         genres_list|          title_list|
+-------+--------------------+------+--------------------+--------------------+--------------------+
|  89008|машина времени в ...|2010.0|Комедии,Фантастик...|[Комедии, Фантаст...|[машина, времени,...|
|  72849|освобождение: огн...|1968.0|Военные,Драмы,Мел...|[Военные, Драмы, ...|[освобождение:, о...|
|  74106|              турист|2011.0|Ужасы,Триллеры,За...|[Ужасы, Триллеры,...|            [турист]|
|  93620|фархат: принц пер...|2009.0|Мультфильмы,Мульт...|[Мультфильмы, Мул...|[фархат:, принц, ...|
|  77690|             бригада|2002.0|Драмы,Русские,Бое...|[Драмы, Русские, ...|           [бригада]|
|  95724|медсестры на задн...|2012.0|  Эротика,Зарубежные|[Эротика, Зарубеж...|[медсестры, на, з...|
|  72605|    волшебники двора|2015.0|Русские мультфиль...|[Русские мультфил...| [волшебники

In [596]:
items = unioned_raw_items.select("item_id", "year", "genres_list", "title_list")

In [597]:
items.show()

+-------+------+--------------------+--------------------+
|item_id|  year|         genres_list|          title_list|
+-------+------+--------------------+--------------------+
|  89008|2010.0|[Комедии, Фантаст...|[машина, времени,...|
|  72849|1968.0|[Военные, Драмы, ...|[освобождение:, о...|
|  74106|2011.0|[Ужасы, Триллеры,...|            [турист]|
|  93620|2009.0|[Мультфильмы, Мул...|[фархат:, принц, ...|
|  77690|2002.0|[Драмы, Русские, ...|           [бригада]|
|  95724|2012.0|[Эротика, Зарубеж...|[медсестры, на, з...|
|  72605|2015.0|[Русские мультфил...| [волшебники, двора]|
|  69574|2014.0|[Драмы, Вестерн, ...|          [спасение]|
|  65908|2016.0|[Драмы, Криминал,...|      [три, девятки]|
|  92916|2008.0|[Русские, Докумен...|         [заложники]|
|  72894|1987.0|[Комедии, Мелодра...|[где, находится, ...|
|   9855|2015.0|[Триллеры, Боевик...|         [ускорение]|
|  83573|1940.0|[Приключения, Бое...|[тимур, и, его, к...|
|   2736|2007.0|[Триллеры, Боевик...|     [приговоренные

In [598]:
items.count()

3704

In [600]:
hasher = HashingTF(inputCol="title_list", outputCol="tf", numFeatures=1000)
tf = hasher.transform(items)
items = IDF(inputCol="tf", outputCol="title_list_features").fit(tf).transform(tf)
items.show(5)

+-------+------+--------------------+--------------------+--------------------+--------------------+
|item_id|  year|         genres_list|          title_list|                  tf| title_list_features|
+-------+------+--------------------+--------------------+--------------------+--------------------+
|  89008|2010.0|[Комедии, Фантаст...|[машина, времени,...|(1000,[328,452,62...|(1000,[328,452,62...|
|  72849|1968.0|[Военные, Драмы, ...|[освобождение:, о...|(1000,[104,443,66...|(1000,[104,443,66...|
|  74106|2011.0|[Ужасы, Триллеры,...|            [турист]|  (1000,[987],[1.0])|(1000,[987],[3.64...|
|  93620|2009.0|[Мультфильмы, Мул...|[фархат:, принц, ...|(1000,[90,236,428...|(1000,[90,236,428...|
|  77690|2002.0|[Драмы, Русские, ...|           [бригада]|  (1000,[203],[1.0])|(1000,[203],[5.73...|
+-------+------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [605]:
cv = CountVectorizer(inputCol="genres_list", outputCol="genres_list_features")
model = cv.fit(items)
items = model.transform(items)

In [607]:
items_feature = items.select("item_id", "year", "title_list_features", "genres_list_features")

In [608]:
items_feature.show()

+-------+------+--------------------+--------------------+
|item_id|  year| title_list_features|genres_list_features|
+-------+------+--------------------+--------------------+
|  89008|2010.0|(1000,[328,452,62...|(83,[0,2,12],[1.0...|
|  72849|1968.0|(1000,[104,443,66...|(83,[1,7,16],[1.0...|
|  74106|2011.0|(1000,[987],[3.64...|(83,[0,3,10],[1.0...|
|  93620|2009.0|(1000,[90,236,428...|(83,[0,14,32],[1....|
|  77690|2002.0|(1000,[203],[5.73...|(83,[1,4,5],[1.0,...|
|  95724|2012.0|(1000,[0,397,665,...|(83,[0,21],[1.0,1...|
|  72605|2015.0|(1000,[828,993],[...|(83,[4,9,15,17,19...|
|  69574|2014.0|(1000,[2],[6.6080...|(83,[0,1,54],[1.0...|
|  65908|2016.0|(1000,[521,741],[...|(83,[0,1,13],[1.0...|
|  92916|2008.0|(1000,[178],[6.13...|(83,[4,27],[1.0,1...|
|  72894|1987.0|(1000,[31,400,778...|(83,[2,6,7],[1.0,...|
|   9855|2015.0|(1000,[205],[6.27...|(83,[0,3,5],[1.0,...|
|  83573|1940.0|(1000,[136,195,39...|(83,[5,8],[1.0,1.0])|
|   2736|2007.0|(1000,[588],[5.81...|(83,[0,3,5],[1.0,..

In [719]:
from pyspark.ml.feature import VectorAssembler

In [720]:
assembler = VectorAssembler(inputCols=["year", "title_list_features", "genres_list_features"], outputCol="item_features")
feature_items = assembler.transform(items)

In [721]:
feture_items.select("item_features").show(5, False)

+--------------------------------------------------------------------------------------------------------------------+
|item_features                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------+
|(1084,[0,663,941,1001,1011],[2016.0,5.5093883366279774,6.020213960393968,1.0,1.0])                                  |
|(1084,[0,18,232,621,1005,1017],[2008.0,5.914853444736141,5.732531887942187,3.064146943232408,1.0,1.0])              |
|(1084,[0,543,616,1003],[2016.0,5.384225193673971,5.819543264931816,1.0])                                            |
|(1084,[0,46,350,1005,1010,1018,1023,1029,1039],[1948.0,5.039384707382242,6.608000625296087,1.0,1.0,1.0,1.0,1.0,1.0])|
|(1084,[0,655,744,1005,1006,1009],[2014.0,5.5093883366279774,4.998562712861986,1.0,1.0,1.0])                         |
+-----------------------------------------------

In [722]:
feature_items = feature_items.select("item_id", "item_features")

In [723]:
feature_items.show()

+-------+--------------------+
|item_id|       item_features|
+-------+--------------------+
|  89008|(1084,[0,329,453,...|
|  72849|(1084,[0,105,444,...|
|  74106|(1084,[0,988,1001...|
|  93620|(1084,[0,91,237,4...|
|  77690|(1084,[0,204,1002...|
|  95724|(1084,[0,1,398,66...|
|  72605|(1084,[0,829,994,...|
|  69574|(1084,[0,3,1001,1...|
|  65908|(1084,[0,522,742,...|
|  92916|(1084,[0,179,1005...|
|  72894|(1084,[0,32,401,7...|
|   9855|(1084,[0,206,1001...|
|  83573|(1084,[0,137,196,...|
|   2736|(1084,[0,589,1001...|
|  72376|(1084,[0,678,1005...|
|  72481|(1084,[0,285,659,...|
|   1704|(1084,[0,202,1003...|
|  66922|(1084,[0,401,634,...|
|  94796|(1084,[0,39,227,6...|
|  67022|(1084,[0,520,529,...|
+-------+--------------------+
only showing top 20 rows



In [724]:
feture_items.count()

3704

In [725]:
df_train.show()

+-------+-------+--------+--------------------+--------------------+
|item_id|user_id|purchase|active_user_purchase|active_item_purchase|
+-------+-------+--------+--------------------+--------------------+
|   8389| 776138|       0|                0.01|        0.0059790732|
|   8389| 833838|       0|         0.005740528|        0.0059790732|
|   8389| 865948|       0|        3.8211694E-4|        0.0059790732|
|   8389| 868207|       0|        3.9277298E-4|        0.0059790732|
|   8389| 896115|       0|        0.0042242706|        0.0059790732|
|   8389| 899680|       0|        3.8022813E-4|        0.0059790732|
|   8389| 902417|       0|        3.8461538E-4|        0.0059790732|
|   8389| 936094|       0|         0.004265219|        0.0059790732|
|   8389| 746917|       0|          3.82263E-4|        0.0059790732|
|   8389| 781373|       0|        3.8153375E-4|        0.0059790732|
|   8389| 797639|       0|                null|        0.0059790732|
|   8389| 890635|       0|        

In [727]:
df_train = df_train.join(feture_items, on="item_id", how="outer")

In [728]:
df_train.show()

+-------+-------+--------+--------------------+--------------------+--------------------+
|item_id|user_id|purchase|active_user_purchase|active_item_purchase|       item_features|
+-------+-------+--------+--------------------+--------------------+--------------------+
|   8389| 793876|       0|        0.0019402406|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 795620|       0|         0.004243827|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 851848|       0|         3.888025E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 880451|       0|         0.009220131|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 900203|       0|        0.0034364262|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 819462|       0|         3.844675E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 834532|       0|        0.0011450382|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 876234|       0|                null|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 

In [731]:
df_train.count()

5032624

In [733]:
df_train.filter(f.isnull("active_user_purchase")).count()

690226

In [735]:
df_train = df_train.na.fill(value=0,subset=["active_user_purchase"])

In [738]:
df_train.show()

+-------+-------+--------+--------------------+--------------------+--------------------+
|item_id|user_id|purchase|active_user_purchase|active_item_purchase|       item_features|
+-------+-------+--------+--------------------+--------------------+--------------------+
|   8389| 793876|       0|        0.0019402406|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 795620|       0|         0.004243827|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 851848|       0|         3.888025E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 880451|       0|         0.009220131|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 900203|       0|        0.0034364262|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 746713|       0|                 0.0|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 883098|       0|         0.001948558|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 903491|       0|        0.0011614402|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 

In [739]:
df_train.filter(f.isnull("active_item_purchase")).count()

835389

In [740]:
df_train = df_train.na.fill(value=0,subset=["active_item_purchase"])

In [741]:
df_train.show()

+-------+-------+--------+--------------------+--------------------+--------------------+
|item_id|user_id|purchase|active_user_purchase|active_item_purchase|       item_features|
+-------+-------+--------+--------------------+--------------------+--------------------+
|   8389| 754230|       0|         0.027575642|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 780033|       0|         7.757952E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 798454|       0|        3.8402458E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 825061|       0|        0.0019312475|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 833685|       0|         0.007500987|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 851486|       0|                 0.0|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 867850|       0|        3.8299503E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 870928|       0|         7.674597E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 

In [742]:
df_train.filter(f.isnull("active_item_purchase")).count()

0

In [743]:
df_train.filter(f.isnull("item_features")).count()

0

In [744]:
assembler = VectorAssembler(inputCols=["active_user_purchase", "active_item_purchase", "item_features"], outputCol="train_features")
train = assembler.transform(df_train)

In [746]:
train.show()

+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+
|item_id|user_id|purchase|active_user_purchase|active_item_purchase|       item_features|      train_features|
+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+
|   8389| 793876|       0|        0.0019402406|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 795620|       0|         0.004243827|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 851848|       0|         3.888025E-4|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 880451|       0|         0.009220131|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 900203|       0|        0.0034364262|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 797350|       0|                 0.0|        0.0059790732|(1084,[0,85,459,6...|(1086,[1,2,87,461...|
|

In [749]:
tarin = train.select("purchase", "train_features")

In [750]:
tarin.show()

+--------+--------------------+
|purchase|      train_features|
+--------+--------------------+
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[1,2,87,461...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
|       0|(1086,[0,1,2,87,4...|
+--------+--------------------+
only showing top 20 rows



In [752]:
from pyspark.ml.classification import GBTClassifier

In [753]:
gbt = GBTClassifier(featuresCol="train_features", labelCol="purchase")

In [754]:
model = gbt.fit(tarin)

In [755]:
model.params

[Param(parent='GBTClassifier_3e91034c48c9', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'),
 Param(parent='GBTClassifier_3e91034c48c9', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'),
 Param(parent='GBTClassifier_3e91034c48c9', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'),
 Param(parent='GBTClassifier_3e91034c48c9', name='featuresCol', doc='features column name'),
 Param(parent='GBTClassifier_3e91034c48c9', name='impurity', doc='Criterion used for information gain calculation (case-

In [726]:
df_test.show()

+-------+-------+--------------------+--------------------+
|item_id|user_id|active_user_purchase|active_item_purchase|
+-------+-------+--------------------+--------------------+
|   8389| 761341|         3.875969E-4|        0.0059790732|
|   8389| 776188|        0.0011525163|        0.0059790732|
|   8389| 846231|        0.0019238169|        0.0059790732|
|   8389| 740836|        0.0027788805|        0.0059790732|
|   8389| 866581|        0.0015917231|        0.0059790732|
|   8389| 901042|        3.8138826E-4|        0.0059790732|
|   8389| 910808|        0.0023781213|        0.0059790732|
|   8389| 923662|                null|        0.0059790732|
|   8389| 928002|        3.8358266E-4|        0.0059790732|
|   8389| 816244|        0.0019230769|        0.0059790732|
|   8389| 899758|          3.85505E-4|        0.0059790732|
|   8389| 927643|        0.0019282685|        0.0059790732|
|   8389| 937273|        0.0022918258|        0.0059790732|
|   8389| 829957|                null|  

In [729]:
df_test = df_test.join(feture_items, on="item_id", how="outer")

In [730]:
df_test.show()

+-------+-------+--------------------+--------------------+--------------------+
|item_id|user_id|active_user_purchase|active_item_purchase|       item_features|
+-------+-------+--------------------+--------------------+--------------------+
|   8389| 822709|         3.789314E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 824008|        3.8211694E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 890476|                null|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 899993|        3.8895372E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 937345|        3.8505968E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 765780|         0.001948558|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 927643|        0.0019282685|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 937273|        0.0022918258|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 806704|        0.0011560693|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 830843|        3.9

In [756]:
df_test = df_test.na.fill(value=0,subset=["active_user_purchase"])

In [757]:
df_test.show()

+-------+-------+--------------------+--------------------+--------------------+
|item_id|user_id|active_user_purchase|active_item_purchase|       item_features|
+-------+-------+--------------------+--------------------+--------------------+
|   8389| 761341|         3.875969E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 776188|        0.0011525163|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 846231|        0.0019238169|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 886063|         0.002698535|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 900335|        0.0046153846|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 936359|         7.613247E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 814235|         7.788162E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 849754|         3.944773E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 860986|        7.6599006E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 887904|        3.8

In [758]:
df_test = df_test.na.fill(value=0,subset=["active_item_purchase"])

In [759]:
df_test.show()

+-------+-------+--------------------+--------------------+--------------------+
|item_id|user_id|active_user_purchase|active_item_purchase|       item_features|
+-------+-------+--------------------+--------------------+--------------------+
|   8389| 761341|         3.875969E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 776188|        0.0011525163|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 846231|        0.0019238169|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 901323|        3.8461538E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 928231|         7.584376E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 816244|        0.0019230769|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 899758|          3.85505E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 824997|        7.7011937E-4|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 837780|        0.0011560693|        0.0059790732|(1084,[0,85,459,6...|
|   8389| 872536|        3.8

In [760]:
df_test.filter(f.isnull("active_user_purchase")).count()

0

In [761]:
df_test.filter(f.isnull("active_item_purchase")).count()

0

In [771]:
assembler = VectorAssembler(inputCols=["active_user_purchase", "active_item_purchase", "item_features"], outputCol="train_features")
test = assembler.transform(df_test)

In [773]:
test.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+
|item_id|user_id|active_user_purchase|active_item_purchase|       item_features|      train_features|
+-------+-------+--------------------+--------------------+--------------------+--------------------+
|   8389| 886063|         0.002698535|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 900335|        0.0046153846|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 936359|         7.613247E-4|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 816244|        0.0019230769|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 899758|          3.85505E-4|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 833749|        3.8595137E-4|        0.0059790732|(1084,[0,85,459,6...|(1086,[0,1,2,87,4...|
|   8389| 869294|        7.6190475E-4|        0.0059790732|(1084,[0,85,459,6...|(1

In [834]:
test = test.select("user_id", "item_id", "train_features")

In [835]:
test.show()

+-------+-------+--------------------+
|user_id|item_id|      train_features|
+-------+-------+--------------------+
| 761341|   8389|(1086,[0,1,2,87,4...|
| 776188|   8389|(1086,[0,1,2,87,4...|
| 846231|   8389|(1086,[0,1,2,87,4...|
| 765780|   8389|(1086,[0,1,2,87,4...|
| 740836|   8389|(1086,[0,1,2,87,4...|
| 866581|   8389|(1086,[0,1,2,87,4...|
| 901042|   8389|(1086,[0,1,2,87,4...|
| 910808|   8389|(1086,[0,1,2,87,4...|
| 923662|   8389|(1086,[1,2,87,461...|
| 928002|   8389|(1086,[0,1,2,87,4...|
| 814235|   8389|(1086,[0,1,2,87,4...|
| 849754|   8389|(1086,[0,1,2,87,4...|
| 860986|   8389|(1086,[0,1,2,87,4...|
| 887904|   8389|(1086,[0,1,2,87,4...|
| 816244|   8389|(1086,[0,1,2,87,4...|
| 899758|   8389|(1086,[0,1,2,87,4...|
| 824997|   8389|(1086,[0,1,2,87,4...|
| 837780|   8389|(1086,[0,1,2,87,4...|
| 872536|   8389|(1086,[0,1,2,87,4...|
| 900450|   8389|(1086,[0,1,2,87,4...|
+-------+-------+--------------------+
only showing top 20 rows



In [840]:
test = test.orderBy(col("user_id"), col("item_id"))

In [841]:
test.show(1000)

+-------+-------+--------------------+
|user_id|item_id|      train_features|
+-------+-------+--------------------+
|   1654|    336|(1086,[0,2,195,67...|
|   1654|    678|(1086,[0,2,361,51...|
|   1654|    691|(1086,[0,2,144,80...|
|   1654|    696|(1086,[0,1,2,6,51...|
|   1654|    763|(1086,[0,1,2,509,...|
|   1654|    795|(1086,[0,1,2,421,...|
|   1654|    861|(1086,[0,1,2,525,...|
|   1654|   1137|(1086,[0,1,2,321,...|
|   1654|   1159|(1086,[0,1,2,26,1...|
|   1654|   1428|(1086,[0,1,2,696,...|
|   1654|   1685|(1086,[0,1,2,202,...|
|   1654|   1686|(1086,[0,1,2,12,4...|
|   1654|   1704|(1086,[0,1,2,204,...|
|   1654|   2093|(1086,[0,2,246,10...|
|   1654|   2343|(1086,[0,1,2,565,...|
|   1654|   2451|(1086,[0,2,571,74...|
|   1654|   2469|(1086,[0,1,2,740,...|
|   1654|   2603|(1086,[0,1,2,995,...|
|   1654|   2609|(1086,[0,2,79,202...|
|   1654|   2621|(1086,[0,1,2,153,...|
|   1654|   2683|(1086,[0,1,2,500,...|
|   1654|   2695|(1086,[0,1,2,984,...|
|   1654|   2696|(1086,[0

In [842]:
df1=test.rdd.zipWithIndex().toDF()

In [843]:
df2=df1.select(col("_1.*"),col("_2").alias('increasing_id'))

In [844]:
df2.show()

+-------+-------+--------------------+-------------+
|user_id|item_id|      train_features|increasing_id|
+-------+-------+--------------------+-------------+
|   1654|    336|(1086,[0,2,195,67...|            0|
|   1654|    678|(1086,[0,2,361,51...|            1|
|   1654|    691|(1086,[0,2,144,80...|            2|
|   1654|    696|(1086,[0,1,2,6,51...|            3|
|   1654|    763|(1086,[0,1,2,509,...|            4|
|   1654|    795|(1086,[0,1,2,421,...|            5|
|   1654|    861|(1086,[0,1,2,525,...|            6|
|   1654|   1137|(1086,[0,1,2,321,...|            7|
|   1654|   1159|(1086,[0,1,2,26,1...|            8|
|   1654|   1428|(1086,[0,1,2,696,...|            9|
|   1654|   1685|(1086,[0,1,2,202,...|           10|
|   1654|   1686|(1086,[0,1,2,12,4...|           11|
|   1654|   1704|(1086,[0,1,2,204,...|           12|
|   1654|   2093|(1086,[0,2,246,10...|           13|
|   1654|   2343|(1086,[0,1,2,565,...|           14|
|   1654|   2451|(1086,[0,2,571,74...|        

In [812]:
df2.count()

2156840

In [845]:
test_features = df2.select("train_features")

In [815]:
test_features.count()

2156840

In [846]:
test_features.show()

+--------------------+
|      train_features|
+--------------------+
|(1086,[0,2,195,67...|
|(1086,[0,2,361,51...|
|(1086,[0,2,144,80...|
|(1086,[0,1,2,6,51...|
|(1086,[0,1,2,509,...|
|(1086,[0,1,2,421,...|
|(1086,[0,1,2,525,...|
|(1086,[0,1,2,321,...|
|(1086,[0,1,2,26,1...|
|(1086,[0,1,2,696,...|
|(1086,[0,1,2,202,...|
|(1086,[0,1,2,12,4...|
|(1086,[0,1,2,204,...|
|(1086,[0,2,246,10...|
|(1086,[0,1,2,565,...|
|(1086,[0,2,571,74...|
|(1086,[0,1,2,740,...|
|(1086,[0,1,2,995,...|
|(1086,[0,2,79,202...|
|(1086,[0,1,2,153,...|
+--------------------+
only showing top 20 rows



In [854]:
predictions = model.transform(test_features)

In [855]:
predictions.show()

+--------------------+--------------------+--------------------+----------+
|      train_features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(1086,[0,2,195,67...|[1.54005601548850...|[0.95606489068482...|       0.0|
|(1086,[0,2,361,51...|[1.54005601548850...|[0.95606489068482...|       0.0|
|(1086,[0,2,144,80...|[1.54005601548850...|[0.95606489068482...|       0.0|
|(1086,[0,1,2,6,51...|[1.53895737887410...|[0.95597250208290...|       0.0|
|(1086,[0,1,2,509,...|[1.54005601548850...|[0.95606489068482...|       0.0|
|(1086,[0,1,2,421,...|[1.50835461524206...|[0.95332331115569...|       0.0|
|(1086,[0,1,2,525,...|[1.54005601548850...|[0.95606489068482...|       0.0|
|(1086,[0,1,2,321,...|[1.53434934556395...|[0.95558297218825...|       0.0|
|(1086,[0,1,2,26,1...|[1.53895737887410...|[0.95597250208290...|       0.0|
|(1086,[0,1,2,696,...|[1.54005601548850...|[0.95606489068482...|       0.0|
|(1086,[0,1,

In [856]:
predictions.show(1, False, True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------
 train_features | (1086,[0,2,195,674,1003,1005,1013,1015],[0.0019470405532047153,2012.0,6.831144176610296,5.3270667798340225,1.0,1.0,1.0,1.0]) 
 rawPrediction  | [1.5400560154885026,-1.5400560154885026]                                                                                     
 probability    | [0.9560648906848211,0.043935109315178855]                                                                                    
 prediction     | 0.0                                                                                                                          
only showing top 1 row



In [862]:
ids = df2.select("increasing_id").rdd.map(lambda x: int(x[0]))

In [863]:
targets = predictions.select("probability").rdd.map(lambda x: float(x[0][1]))

In [864]:
submission = spark.createDataFrame(ids.zip(targets), StructType([StructField("increasing_id", IntegerType(), True), StructField("purchase", FloatType(), True)]))

In [865]:
submission.show()

+-------------+-----------+
|increasing_id|   purchase|
+-------------+-----------+
|            0| 0.04393511|
|            1| 0.04393511|
|            2| 0.04393511|
|            3|0.044027496|
|            4| 0.04393511|
|            5|0.046676688|
|            6| 0.04393511|
|            7|0.044417027|
|            8|0.044027496|
|            9| 0.04393511|
|           10|0.044027496|
|           11| 0.04393511|
|           12|  0.0441698|
|           13| 0.04393511|
|           14| 0.04393511|
|           15| 0.04393511|
|           16| 0.04704974|
|           17| 0.04393511|
|           18| 0.04393511|
|           19|0.044027496|
+-------------+-----------+
only showing top 20 rows



In [866]:
unic_submission = df2.join(submission, on="increasing_id", how="inner")

In [867]:
unic_submission.show()

+-------------+-------+-------+--------------------+-----------+
|increasing_id|user_id|item_id|      train_features|   purchase|
+-------------+-------+-------+--------------------+-----------+
|           26|   1654|   3055|(1086,[0,2,545,60...| 0.04393511|
|           29|   1654|   3147|(1086,[0,1,2,316,...|0.044027496|
|          474|   1654|  74689|(1086,[0,1,2,204,...| 0.04393511|
|          964|   1654|  95279|(1086,[0,2,593,10...| 0.04393511|
|         1677| 510087|  79033|(1086,[0,1,2,123,...|0.045030724|
|         1697| 510087|  79837|(1086,[0,1,2,492,...|0.055638693|
|         1806| 510087|  87536|(1086,[0,1,2,185,...|0.044132076|
|         1950| 510087|  92826|(1086,[0,2,119,43...|0.044132076|
|         2040| 510087|  94143|(1086,[0,1,2,469,...|0.044132076|
|         2214| 510087|  99925|(1086,[0,1,2,540,...|0.044132076|
|         2250| 510087| 100384|(1086,[0,1,2,131,...|0.044132076|
|         2453| 517612|   9717|(1086,[0,1,2,304,...| 0.04383287|
|         2509| 517612|  

In [868]:
unic_submission.count()

2156840

In [869]:
sub_sort = unic_submission.orderBy(col("user_id"), col("item_id"))

In [870]:
sub_sort.show()

+-------------+-------+-------+--------------------+-----------+
|increasing_id|user_id|item_id|      train_features|   purchase|
+-------------+-------+-------+--------------------+-----------+
|            0|   1654|    336|(1086,[0,2,195,67...| 0.04393511|
|            1|   1654|    678|(1086,[0,2,361,51...| 0.04393511|
|            2|   1654|    691|(1086,[0,2,144,80...| 0.04393511|
|            3|   1654|    696|(1086,[0,1,2,6,51...|0.044027496|
|            4|   1654|    763|(1086,[0,1,2,509,...| 0.04393511|
|            5|   1654|    795|(1086,[0,1,2,421,...|0.046676688|
|            6|   1654|    861|(1086,[0,1,2,525,...| 0.04393511|
|            7|   1654|   1137|(1086,[0,1,2,321,...|0.044417027|
|            8|   1654|   1159|(1086,[0,1,2,26,1...|0.044027496|
|            9|   1654|   1428|(1086,[0,1,2,696,...| 0.04393511|
|           10|   1654|   1685|(1086,[0,1,2,202,...|0.044027496|
|           11|   1654|   1686|(1086,[0,1,2,12,4...| 0.04393511|
|           12|   1654|  

In [873]:
sub = sub_sort.select(col("increasing_id").alias(""), col("user_id"), col("item_id"), col("purchase"))

In [901]:
new_sub = sub.repartition(1)

In [902]:
new_sub.show()

+-----+-------+-------+-----------+
|     |user_id|item_id|   purchase|
+-----+-------+-------+-----------+
|20953| 636572|  94988|0.051238086|
|20954| 636572|  94989|0.043697894|
|20955| 636572|  94993|  0.0438097|
|20956| 636572|  94995| 0.04383287|
|20957| 636572|  94996|0.043697894|
|20958| 636572|  95000| 0.04383287|
|20959| 636572|  95004| 0.04383287|
|20960| 636572|  95011| 0.04383287|
|20961| 636572|  95013|0.043690737|
|20962| 636572|  95017|0.043690737|
|20963| 636572|  95023|0.043690737|
|20964| 636572|  95041| 0.04395095|
|20965| 636572|  95042|0.043942727|
|20966| 636572|  95044|0.043697894|
|20967| 636572|  95054|0.043690737|
|20968| 636572|  95056|0.043690737|
|20969| 636572|  95057|0.043697894|
|20970| 636572|  95064|0.043697894|
|20971| 636572|  95065| 0.04395095|
|20972| 636572|  95073| 0.04395095|
+-----+-------+-------+-----------+
only showing top 20 rows



In [903]:
new_sub.rdd.getNumPartitions()

1

In [904]:
new_sub_sort = new_sub.orderBy(col("user_id"), col("item_id"))

In [905]:
new_sub_sort.show()

+---+-------+-------+-----------+
|   |user_id|item_id|   purchase|
+---+-------+-------+-----------+
|  0|   1654|    336| 0.04393511|
|  1|   1654|    678| 0.04393511|
|  2|   1654|    691| 0.04393511|
|  3|   1654|    696|0.044027496|
|  4|   1654|    763| 0.04393511|
|  5|   1654|    795|0.046676688|
|  6|   1654|    861| 0.04393511|
|  7|   1654|   1137|0.044417027|
|  8|   1654|   1159|0.044027496|
|  9|   1654|   1428| 0.04393511|
| 10|   1654|   1685|0.044027496|
| 11|   1654|   1686| 0.04393511|
| 12|   1654|   1704|  0.0441698|
| 13|   1654|   2093| 0.04393511|
| 14|   1654|   2343| 0.04393511|
| 15|   1654|   2451| 0.04393511|
| 16|   1654|   2469| 0.04704974|
| 17|   1654|   2603| 0.04393511|
| 18|   1654|   2609| 0.04393511|
| 19|   1654|   2621|0.044027496|
+---+-------+-------+-----------+
only showing top 20 rows



In [910]:
predictions_pd = new_sub_sort.toPandas()

In [913]:
!pwd

/data/home/darina.bocharova


In [918]:
predictions_pd.to_csv('lab03.csv', header=True, index= False )

In [897]:
#sub.write.csv("lab03.csv")

AnalysisException: 'path hdfs://spark-master-1.newprolab.com:8020/user/darina.bocharova/lab03.csv already exists.;'

# views_programmes

In [423]:
!hdfs dfs -head /labs/slaba03/laba03_views_programmes.csv

user_id,item_id,ts_start,ts_end,item_type
0,7101053,1491409931,1491411600,live
0,7101054,1491412481,1491451571,live
0,7101054,1491411640,1491412481,live
0,6184414,1486191290,1486191640,live
257,4436877,1490628499,1490630256,live
1654,7489015,1493434801,1493435401,live
1654,7489023,1493444101,1493445601,live
1654,6617053,1489186156,1489200834,live
1654,6438693,1487840070,1487840433,live
1654,6526859,1488705452,1488706154,live
1654,6526754,1488532396,1488532895,pvr
1654,6239098,1486732011,1486732410,live
1654,6438763,1488305761,1488307286,pvr
1654,7489013,1493433301,1493434201,live
1654,6317094,1486829784,1486830389,live
1654,6799393,1490172025,1490173391,pvr
1654,6616978,1488962050,1488962874,pvr
1654,6384471,1487332434,1487333704,live
1654,6688552,1489602627,1489603198,pvr
1654,6678411,1489307460,1489307856,live
1654,7489002,1493409601,1493413201,live
1654,6275396,1486717960,1486718575,live
1654,7528428,1493629201,1493629580,live
1654,7474559,1493392251,1493392757,live
1654,6290030,148

In [424]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType, FloatType

In [425]:
schema_views = StructType([
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("ts_start", IntegerType()),
    StructField("ts_end", IntegerType()),
    StructField("item_type", StringType())
])

In [426]:
raw_views = spark.read.csv("/labs/slaba03/laba03_views_programmes.csv", sep=",", schema=schema_views, header=True)

In [427]:
raw_views.show()

+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
|      0|7101053|1491409931|1491411600|     live|
|      0|7101054|1491412481|1491451571|     live|
|      0|7101054|1491411640|1491412481|     live|
|      0|6184414|1486191290|1486191640|     live|
|    257|4436877|1490628499|1490630256|     live|
|   1654|7489015|1493434801|1493435401|     live|
|   1654|7489023|1493444101|1493445601|     live|
|   1654|6617053|1489186156|1489200834|     live|
|   1654|6438693|1487840070|1487840433|     live|
|   1654|6526859|1488705452|1488706154|     live|
|   1654|6526754|1488532396|1488532895|      pvr|
|   1654|6239098|1486732011|1486732410|     live|
|   1654|6438763|1488305761|1488307286|      pvr|
|   1654|7489013|1493433301|1493434201|     live|
|   1654|6317094|1486829784|1486830389|     live|
|   1654|6799393|1490172025|1490173391|      pvr|
|   1654|6616978|1488962050|1488962874|      pvr|


In [428]:
raw_views.count()

20845607

In [429]:
raw_views.rdd.getNumPartitions()

7

In [430]:
raw_views.filter(f.isnull("item_type")).count()

0

In [431]:
raw_views.select('user_id').distinct().count()

79385

In [432]:
raw_views.select('item_id').distinct().count()

633840

In [433]:
raw_views.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- ts_start: integer (nullable = true)
 |-- ts_end: integer (nullable = true)
 |-- item_type: string (nullable = true)



In [444]:
unic_user_id = raw_train.select('user_id').distinct()
unic_user_id.show()

+-------+
|user_id|
+-------+
| 776188|
| 761341|
| 833685|
| 825061|
| 780033|
| 798454|
| 754230|
| 870928|
| 846231|
| 851486|
| 867850|
| 879401|
| 901457|
| 928140|
| 927211|
| 793876|
| 822709|
| 795620|
| 824008|
| 851848|
+-------+
only showing top 20 rows



In [445]:
unic_user_id.count()

1941

In [446]:
unic_user_id_views = unic_user_id.join(raw_views, on="user_id", how="inner")

In [447]:
unic_user_id_views.show()

+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
| 754230|7390400|1492977466|1492981324|      pvr|
| 754230|7336221|1492939870|1492940429|     live|
| 754230|7478327|1493186401|1493187301|     live|
| 754230|7458006|1493380501|1493381468|     live|
| 754230|6687800|1489560537|1489565587|     live|
| 754230|6605919|1489311836|1489312182|     live|
| 754230|6625274|1489215142|1489216945|      pvr|
| 754230|6811570|1490549199|1490550706|      pvr|
| 754230|6706194|1489641332|1489643447|      pvr|
| 754230|6357609|1487265316|1487268028|     live|
| 754230|6169532|1486298888|1486300691|      pvr|
| 754230|6959510|1490955136|1490955797|     live|
| 754230|6834963|1490504911|1490506320|     live|
| 754230|6444416|1487862335|1487863850|      pvr|
| 754230|6922552|1491061704|1491063761|     live|
| 754230|6706265|1489762677|1489764443|      pvr|
| 754230|6357713|1487505677|1487507143|      pvr|


In [448]:
unic_user_id_views.select("user_id").distinct().count()

1938

In [456]:
list1_unic_user_id = unic_user_id.collect()

In [457]:
len(list1_unic_user_id)

1941

In [458]:
list1_unic_user_id[1][0]

761341

In [459]:
tot_list1_unic_user_id = []
for i in range(len(list1_unic_user_id)):
    tot_list1_unic_user_id.append(list1_unic_user_id[i][0])

In [460]:
len(tot_list1_unic_user_id)

1941

In [461]:
tot_list1_unic_user_id

[776188,
 761341,
 833685,
 825061,
 780033,
 798454,
 754230,
 870928,
 846231,
 851486,
 867850,
 879401,
 901457,
 928140,
 927211,
 793876,
 822709,
 795620,
 824008,
 851848,
 890476,
 880451,
 899993,
 900203,
 937345,
 746713,
 566758,
 816426,
 892290,
 883098,
 903826,
 916566,
 903491,
 886063,
 900335,
 936359,
 920599,
 899224,
 901323,
 932980,
 928231,
 939459,
 834532,
 819462,
 876234,
 913290,
 928490,
 765780,
 861601,
 842784,
 861707,
 894860,
 797350,
 740836,
 866581,
 848246,
 902359,
 901042,
 910808,
 928002,
 923662,
 814235,
 778082,
 787896,
 844427,
 842883,
 849754,
 860986,
 887904,
 799148,
 816244,
 853717,
 871607,
 897570,
 899758,
 824997,
 857661,
 837780,
 872536,
 918664,
 900450,
 925047,
 833749,
 869294,
 894833,
 895407,
 906790,
 920718,
 927261,
 751096,
 785772,
 627053,
 830409,
 811441,
 841902,
 869418,
 849940,
 881844,
 935828,
 925845,
 923003,
 852684,
 868695,
 853468,
 891380,
 895034,
 877109,
 905722,
 915444,
 905635,
 926143,
 

In [465]:
list2_unic_user_id = unic_user_id_views.select("user_id").distinct()
list2_unic_user_id.show()

+-------+
|user_id|
+-------+
| 754230|
| 761341|
| 776188|
| 780033|
| 798454|
| 825061|
| 833685|
| 846231|
| 851486|
| 867850|
| 870928|
| 879401|
| 901457|
| 927211|
| 928140|
| 793876|
| 795620|
| 822709|
| 824008|
| 851848|
+-------+
only showing top 20 rows



In [467]:
list2_unic_user_id.count()

1938

In [469]:
list2_unic_user_id = list2_unic_user_id.collect()

In [470]:
tot_list2_unic_user_id = []
for i in range(len(list2_unic_user_id)):
    tot_list2_unic_user_id.append(list2_unic_user_id[i][0])

In [471]:
len(tot_list2_unic_user_id)

1938

In [472]:
tot_list2_unic_user_id

[754230,
 761341,
 776188,
 780033,
 798454,
 825061,
 833685,
 846231,
 851486,
 867850,
 870928,
 879401,
 901457,
 927211,
 928140,
 793876,
 795620,
 822709,
 824008,
 851848,
 880451,
 890476,
 899993,
 900203,
 937345,
 566758,
 746713,
 816426,
 883098,
 892290,
 903491,
 903826,
 916566,
 886063,
 900335,
 920599,
 936359,
 899224,
 901323,
 928231,
 932980,
 939459,
 819462,
 834532,
 876234,
 913290,
 928490,
 765780,
 842784,
 861601,
 861707,
 894860,
 740836,
 797350,
 848246,
 866581,
 901042,
 902359,
 910808,
 923662,
 928002,
 778082,
 787896,
 814235,
 842883,
 844427,
 849754,
 860986,
 887904,
 799148,
 816244,
 853717,
 871607,
 897570,
 899758,
 824997,
 837780,
 857661,
 872536,
 900450,
 918664,
 925047,
 833749,
 869294,
 894833,
 895407,
 906790,
 920718,
 927261,
 627053,
 751096,
 785772,
 811441,
 830409,
 841902,
 849940,
 869418,
 881844,
 923003,
 925845,
 935828,
 852684,
 853468,
 868695,
 877109,
 891380,
 895034,
 905635,
 905722,
 915444,
 921046,
 

In [475]:
l1 = tot_list1_unic_user_id
l2 = tot_list2_unic_user_id

res = [x for x in l1 + l2 if x not in l1 or x not in l2]
res 

[863483, 902845, 901918]

In [512]:
unic_user_id_views.show()

+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
| 754230|7390400|1492977466|1492981324|      pvr|
| 754230|7336221|1492939870|1492940429|     live|
| 754230|7478327|1493186401|1493187301|     live|
| 754230|7458006|1493380501|1493381468|     live|
| 754230|6687800|1489560537|1489565587|     live|
| 754230|6605919|1489311836|1489312182|     live|
| 754230|6625274|1489215142|1489216945|      pvr|
| 754230|6811570|1490549199|1490550706|      pvr|
| 754230|6706194|1489641332|1489643447|      pvr|
| 754230|6357609|1487265316|1487268028|     live|
| 754230|6169532|1486298888|1486300691|      pvr|
| 754230|6959510|1490955136|1490955797|     live|
| 754230|6834963|1490504911|1490506320|     live|
| 754230|6444416|1487862335|1487863850|      pvr|
| 754230|6922552|1491061704|1491063761|     live|
| 754230|6706265|1489762677|1489764443|      pvr|
| 754230|6357713|1487505677|1487507143|      pvr|


## views_programmes item_id не пересекаются с train, test

In [480]:
unic_item_id = raw_train.select('item_id').distinct()
unic_item_id.show()

+-------+
|item_id|
+-------+
|  74820|
|  90019|
|   8638|
|  99817|
|  93486|
|  95080|
|  74757|
|  95940|
|  94819|
|  94851|
|  78113|
|  10817|
|  72820|
|   8389|
|  10914|
|   1522|
|  93633|
|  74440|
|  73041|
|  94621|
+-------+
only showing top 20 rows



In [481]:
unic_item_id.count()

3704

In [484]:
unic_item_id.show()

+-------+
|item_id|
+-------+
|  74820|
|  90019|
|   8638|
|  99817|
|  93486|
|  95080|
|  74757|
|  95940|
|  94819|
|  94851|
|  78113|
|  10817|
|  72820|
|   8389|
|  10914|
|   1522|
|  93633|
|  74440|
|  73041|
|  94621|
+-------+
only showing top 20 rows



In [486]:
list1_unic_item_id = unic_item_id.collect()

In [514]:
len(list1_unic_item_id)

3704

In [495]:
tot_list1_unic_item_id = []
for i in range(len(list1_unic_item_id)):
    tot_list1_unic_item_id.append(list1_unic_item_id[i][0])

In [516]:
len(tot_list1_unic_item_id)

3704

In [499]:
print(sorted(tot_list1_unic_item_id))

[326, 336, 357, 396, 400, 423, 430, 449, 453, 478, 495, 505, 540, 546, 578, 592, 595, 601, 678, 683, 691, 696, 728, 751, 752, 754, 763, 781, 782, 786, 795, 802, 861, 902, 936, 941, 948, 981, 1125, 1129, 1131, 1132, 1137, 1151, 1159, 1222, 1258, 1307, 1320, 1345, 1428, 1477, 1522, 1525, 1599, 1685, 1686, 1704, 1738, 1778, 1789, 1861, 1867, 1870, 1875, 1912, 2027, 2051, 2058, 2072, 2076, 2093, 2105, 2136, 2166, 2222, 2252, 2304, 2313, 2327, 2328, 2343, 2349, 2360, 2364, 2405, 2406, 2451, 2469, 2562, 2603, 2607, 2609, 2621, 2632, 2683, 2695, 2696, 2705, 2736, 2773, 2785, 2809, 2830, 2832, 2855, 2951, 2967, 2990, 2999, 3055, 3066, 3083, 3085, 3099, 3110, 3122, 3142, 3147, 3148, 3159, 3176, 3182, 3188, 3195, 3232, 3250, 3253, 3267, 3294, 3298, 3305, 3314, 3408, 3470, 3481, 3515, 3587, 3625, 3630, 3646, 3686, 3692, 3694, 3703, 3711, 3722, 3724, 3750, 3751, 3754, 3755, 3759, 3764, 3811, 3821, 3823, 3838, 3851, 3878, 3894, 3895, 3903, 3991, 4017, 4033, 4120, 4209, 4238, 4253, 4255, 4283, 4302,

In [485]:
raw_views.show()

+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
|      0|7101053|1491409931|1491411600|     live|
|      0|7101054|1491412481|1491451571|     live|
|      0|7101054|1491411640|1491412481|     live|
|      0|6184414|1486191290|1486191640|     live|
|    257|4436877|1490628499|1490630256|     live|
|   1654|7489015|1493434801|1493435401|     live|
|   1654|7489023|1493444101|1493445601|     live|
|   1654|6617053|1489186156|1489200834|     live|
|   1654|6438693|1487840070|1487840433|     live|
|   1654|6526859|1488705452|1488706154|     live|
|   1654|6526754|1488532396|1488532895|      pvr|
|   1654|6239098|1486732011|1486732410|     live|
|   1654|6438763|1488305761|1488307286|      pvr|
|   1654|7489013|1493433301|1493434201|     live|
|   1654|6317094|1486829784|1486830389|     live|
|   1654|6799393|1490172025|1490173391|      pvr|
|   1654|6616978|1488962050|1488962874|      pvr|


In [502]:
list2_unic_item_id = raw_views.select(raw_views.item_id)

In [503]:
list2_unic_item_id.show()

+-------+
|item_id|
+-------+
|7101053|
|7101054|
|7101054|
|6184414|
|4436877|
|7489015|
|7489023|
|6617053|
|6438693|
|6526859|
|6526754|
|6239098|
|6438763|
|7489013|
|6317094|
|6799393|
|6616978|
|6384471|
|6688552|
|6678411|
+-------+
only showing top 20 rows



In [504]:
list2_unic_item_id = list2_unic_item_id.distinct().collect()

In [507]:
len(list2_unic_item_id)

633840

In [508]:
tot_list2_unic_item_id = []
for i in range(len(list2_unic_item_id)):
    tot_list2_unic_item_id.append(list2_unic_item_id[i][0])

In [517]:
len(tot_list2_unic_item_id)

633840

In [511]:
sorted(tot_list2_unic_item_id)

[-99996,
 -1,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 50,
 51,
 52,
 54,
 56,
 57,
 58,
 59,
 60,
 63,
 64,
 67,
 68,
 69,
 72,
 76,
 80,
 84,
 88,
 89,
 120,
 4436877,
 6053729,
 6054224,
 6054232,
 6054233,
 6054236,
 6054237,
 6060166,
 6060332,
 6060333,
 6060334,
 6060335,
 6060336,
 6060337,
 6060338,
 6060339,
 6060340,
 6066740,
 6066742,
 6070763,
 6070766,
 6070767,
 6070768,
 6070769,
 6070770,
 6070772,
 6070773,
 6070774,
 6070862,
 6070865,
 6070866,
 6070868,
 6071454,
 6071458,
 6071460,
 6075465,
 6076249,
 6076250,
 6077666,
 6077677,
 6078517,
 6078519,
 6079771,
 6081271,
 6081275,
 6081283,
 6082675,
 6082676,
 6082677,
 6082678,
 6082679,
 6082680,
 6082841,
 6083653,
 6083654,
 6083655,
 6083656,
 6083657,
 6083658,
 6083659,
 6083666,
 6084165,
 6086146,
 6086515,
 6086520,

In [482]:
unic_item_id_views = unic_item_id.join(raw_views, on="item_id", how="inner")

In [483]:
unic_item_id_views.show()

+-------+-------+--------+------+---------+
|item_id|user_id|ts_start|ts_end|item_type|
+-------+-------+--------+------+---------+
+-------+-------+--------+------+---------+



In [406]:
raw_views.filter(raw_views.item_id == 94621).show()

+-------+-------+--------+------+---------+
|user_id|item_id|ts_start|ts_end|item_type|
+-------+-------+--------+------+---------+
+-------+-------+--------+------+---------+



In [407]:
raw_views.filter(raw_views.item_id == -99996).show()

+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
| 768480| -99996|1491411022|1491411600|     live|
+-------+-------+----------+----------+---------+



In [410]:
raw_views.filter("item_id LIKE '%-%'").show(1000)

+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
| 731037|     -1|1490983423|1490984058|     live|
| 731037|     -1|1490987413|1490998819|     live|
| 731037|     -1|1490984073|1490987402|     live|
| 731037|     -1|1491058394|1491064761|     live|
| 744719|     -1|1489591648|1489606401|     live|
| 745335|     -1|1491125626|1491130500|     live|
| 745335|     -1|1491130500|1491132137|     live|
| 747028|     -1|1489900260|1489900818|     live|
| 765780|     -1|1493249885|1493256940|     live|
| 768480| -99996|1491411022|1491411600|     live|
| 787708|     -1|1489657449|1489658124|     live|
| 820904|     -1|1490845456|1490860158|     live|
| 850009|     -1|1488994800|1489042691|     live|
| 850009|     -1|1488993208|1488994800|     live|
| 853052|     -1|1491512400|1491512932|     live|
| 853140|     -1|1493750950|1493752258|     live|
| 854248|     -1|1490977380|1490988917|     live|


In [358]:
unic_item_id_views.show()

+-------+-------+--------+------+---------+
|item_id|user_id|ts_start|ts_end|item_type|
+-------+-------+--------+------+---------+
+-------+-------+--------+------+---------+



In [375]:
raw_views_list = raw_views.select("item_id").distinct().collect()

In [376]:
raw_views_list

[Row(item_id=6526786),
 Row(item_id=7076695),
 Row(item_id=6604291),
 Row(item_id=6826302),
 Row(item_id=6963158),
 Row(item_id=6687869),
 Row(item_id=6268253),
 Row(item_id=6247754),
 Row(item_id=6267729),
 Row(item_id=6688668),
 Row(item_id=6944928),
 Row(item_id=6280644),
 Row(item_id=6734233),
 Row(item_id=7369899),
 Row(item_id=6968171),
 Row(item_id=6816863),
 Row(item_id=6328525),
 Row(item_id=6833183),
 Row(item_id=6643858),
 Row(item_id=6379275),
 Row(item_id=6144637),
 Row(item_id=6538790),
 Row(item_id=7566327),
 Row(item_id=7108034),
 Row(item_id=6600601),
 Row(item_id=6600565),
 Row(item_id=6600567),
 Row(item_id=7111621),
 Row(item_id=6550191),
 Row(item_id=7352951),
 Row(item_id=6599753),
 Row(item_id=6586891),
 Row(item_id=6599514),
 Row(item_id=6781667),
 Row(item_id=6467228),
 Row(item_id=6687704),
 Row(item_id=6922554),
 Row(item_id=6279182),
 Row(item_id=6688023),
 Row(item_id=6328777),
 Row(item_id=6827938),
 Row(item_id=6520838),
 Row(item_id=7390363),
 Row(item_i

In [398]:
views_item_id_list = []

In [399]:
[views_item_id_list.append(raw_views_list[i][0]) for i in range(len(raw_views_list))]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [402]:
len(views_item_id_list)

633840

In [403]:
min(views_item_id_list)

-99996

In [404]:
max(views_item_id_list)

7610659

In [411]:
views_item_id_list.remove(-1)

In [412]:
views_item_id_list.remove(-99996)

In [413]:
min(views_item_id_list)

1

In [414]:
max(views_item_id_list)

7610659

In [919]:
spark.stop()

In [920]:
user_jsonuser_jsonuser_json

AttributeError: 'NoneType' object has no attribute 'sc'